# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-bf886f15f6-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Author comparison explorative search") 

Consider the following exploratory scenario:


>  Investigate Italian and French book authors in terms of awards, books published and copyright types



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P17`     | country       | predicate | 
| `wdt:P27`     | citizenship   | predicate | 
| `wd:Q36180`   | writer        | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q172579`  | Kingdom of Italy        | node |
| `wd:Q142`     | France        | node |
| `wd:Q37922`   | Nobel Prize literature        | node |
| `wd:Q213678`  | Vatican Library        | node |


Also consider that

```
?p wdt:P27 wd:Q142
```

is the BGP to retrieve all **French citizens**


The workload should


1. Identify the BGP for obtaining the Italian and French writers who published a book in the last 50 years

2. Compare the number of books written by Italian and French writers

3. Count how many books written by Italian authors are now released with a "public domain" copyright form

4. How many Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 

5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(?p)
WHERE { 
?p wdt:P27 wd:Q142 .
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '273456')]


1

#### Identify the BGP for obtaining the Italian and French writers who published a book in the last 50 years

In [4]:
# First search for citiziens and their jobs in france


queryString = """
SELECT *
WHERE { 
    ?p wdt:P27 wd:Q142 .
    ?p <http://schema.org/name> ?name .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)


## khob in query faghat citizen haye keshvar france ro neshon mide
# ye joori hala search kon k ye shakhs faransavi ro peyda koni 


Results
[('p', 'http://www.wikidata.org/entity/Q3047923'), ('name', 'Edmond Maître')]
[('p', 'http://www.wikidata.org/entity/Q310715'), ('name', 'Henri Fantin-Latour')]
[('p', 'http://www.wikidata.org/entity/Q1566596'), ('name', 'Nicolas Flamel')]
[('p', 'http://www.wikidata.org/entity/Q3106353'), ('name', 'Gilles Legrand')]
[('p', 'http://www.wikidata.org/entity/Q3169515'), ('name', 'Jean-Pierre Limosin')]
[('p', 'http://www.wikidata.org/entity/Q328926'), ('name', 'Philippe Ramos')]
[('p', 'http://www.wikidata.org/entity/Q584886'), ('name', 'François Villiers')]
[('p', 'http://www.wikidata.org/entity/Q742799'), ('name', 'Philippe Lioret')]
[('p', 'http://www.wikidata.org/entity/Q936929'), ('name', 'Pascal Laugier')]
[('p', 'http://www.wikidata.org/entity/Q3120486'), ('name', 'William I of Baux')]


10

In [5]:
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 
    wd:Q3120486 ?p ?o .
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)
# this showed all the infromation related to a person

Results
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pname', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q8688594'), ('oname', 'Hugo III of Baux')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('o', 'http://www.wikidata.org/entity/Q12344159'), ('oname', 'William')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q150'), ('oname', 'French')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q186370'), ('oname', 'troubadour')]
[('p', 'http://www.wikidata.org/prop/direct/P97'), ('pname', 'noble title'), ('o', 'http://www.wikidata.org/entity/Q2747456'), ('oname', 'prince')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 

18

In [6]:
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 
    wd:Q3120486 wdt:P106 ?o .
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P348'), ('pname', 'software version identifier'), ('o', 'http://www.wikidata.org/entity/Q186370'), ('oname', 'troubadour')]
[('p', 'http://www.wikidata.org/entity/P4669'), ('pname', 'tabular software version'), ('o', 'http://www.wikidata.org/entity/Q186370'), ('oname', 'troubadour')]
[('p', 'http://www.wikidata.org/entity/P747'), ('pname', 'has edition or translation'), ('o', 'http://www.wikidata.org/entity/Q186370'), ('oname', 'troubadour')]
[('p', 'http://www.wikidata.org/entity/Q1002728'), ('pname', 'Fonsi Nieto'), ('o', 'http://www.wikidata.org/entity/Q186370'), ('oname', 'troubadour')]
[('p', 'http://www.wikidata.org/entity/Q100326534'), ('pname', "Al Stewart's albums in chronological order"), ('o', 'http://www.wikidata.org/entity/Q186370'), ('oname', 'troubadour')]
[('p', 'http://www.wikidata.org/entity/Q100327667'), ('pname', 'Category:Adelina Lopes Vieira'), ('o', 'http://www.wikidata.org/entity/Q186370'), ('oname', 'troubadour')]


20

In [7]:
## All writers # french

queryString = """
SELECT ?p ?pname 
WHERE { 
    ?p wdt:P27 wd:Q142 .
    ?p wdt:P106 wd:Q36180 .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q135640'), ('pname', 'Valery Larbaud')]
[('p', 'http://www.wikidata.org/entity/Q11945240'), ('pname', 'Renat Lavaud')]
[('p', 'http://www.wikidata.org/entity/Q1225448'), ('pname', 'Georges Lafenestre')]
[('p', 'http://www.wikidata.org/entity/Q1248117'), ('pname', 'René Doumic')]
[('p', 'http://www.wikidata.org/entity/Q18057423'), ('pname', 'Maurice Dreyfous')]
[('p', 'http://www.wikidata.org/entity/Q2741209'), ('pname', 'Téodor de Wyzewa')]
[('p', 'http://www.wikidata.org/entity/Q523039'), ('pname', 'André Theuriet')]
[('p', 'http://www.wikidata.org/entity/Q589611'), ('pname', 'Philippe Chatrier')]
[('p', 'http://www.wikidata.org/entity/Q669685'), ('pname', 'Robert de Montesquiou')]
[('p', 'http://www.wikidata.org/entity/Q733214'), ('pname', 'Bonaventure des Périers')]
[('p', 'http://www.wikidata.org/entity/Q77742'), ('pname', 'Gottlieb Konrad Pfeffel')]
[('p', 'http://www.wikidata.org/entity/Q1210514'), ('pname', 'Didier Barbelivien')]
[(

50

In [8]:
#Q1596281 Marcellin Desboutin
# search for book property
queryString = """
SELECT ?s ?sname ?p ?pname 
WHERE { 
    ?s ?p wd:Q1596281 .
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q17492504'), ('sname', 'The Baby Carriage'), ('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator')]
[('s', 'http://www.wikidata.org/entity/Q1239950'), ('sname', "L'Absinthe"), ('p', 'http://www.wikidata.org/prop/direct/P180'), ('pname', 'depicts')]
[('s', 'http://www.wikidata.org/entity/Q16399276'), ('sname', '2nd impressionist exhibition'), ('p', 'http://www.wikidata.org/prop/direct/P710'), ('pname', 'participant')]
[('s', 'http://www.wikidata.org/entity/Q43294256'), ('sname', 'Portrait of Marcellin Desboutin'), ('p', 'http://www.wikidata.org/prop/direct/P180'), ('pname', 'depicts')]
[('s', 'http://www.wikidata.org/entity/Q43294256'), ('sname', 'Portrait of Marcellin Desboutin'), ('p', 'http://www.wikidata.org/prop/direct/P921'), ('pname', 'main subject')]
[('s', 'http://www.wikidata.org/entity/Q65080945'), ('sname', 'Degas'), ('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator')]
[('s', 'http://www.wik

19

In [9]:
#Q1596281 Marcellin Desboutin

queryString = """
SELECT ?s ?sname ?p ?pname 
WHERE { 
    wd:Q1596281 ?s ?p .
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P103'), ('sname', 'native language'), ('p', 'http://www.wikidata.org/entity/Q150'), ('pname', 'French')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q36180'), ('pname', 'writer')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q1028181'), ('pname', 'painter')]
[('s', 'http://www.wikidata.org/prop/direct/P1343'), ('sname', 'described by source'), ('p', 'http://www.wikidata.org/entity/Q67311526'), ('pname', 'Obalky knih.cz')]
[('s', 'http://www.wikidata.org/prop/direct/P136'), ('sname', 'genre'), ('p', 'http://www.wikidata.org/entity/Q134307'), ('pname', 'portrait')]
[('s', 'http://www.wikidata.org/prop/direct/P1412'), ('sname', 'languages spoken, written or signed'), ('p', 'http://www.wikidata.org/entity/Q150'), ('pname', 'French')]
[('s', 'http://www.wikidata.org/prop/direct/P166'), ('sname', 'awar

36

In [10]:
#Q1596281 Marcellin Desboutin

queryString = """
SELECT ?p ?pname 
WHERE { 
    wd:Q1596281 wdt:P800 ?p .
    ?p <http://schema.org/name> ?pname .

} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q17492504'), ('pname', 'The Baby Carriage')]


1

In [11]:
# Category:André Theuriet Q15891399
# search for book property in another author node
queryString = """
SELECT ?s ?sname ?p ?pname 
WHERE { 
    wd:Q15891399 ?s ?p .
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)


Results
[('s', 'http://www.wikidata.org/prop/direct/P301'), ('sname', "category's main topic"), ('p', 'http://www.wikidata.org/entity/Q523039'), ('pname', 'André Theuriet')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q4167836'), ('pname', 'Wikimedia category')]


2

In [12]:
# Category:André Theuriet Q15891399

queryString = """
SELECT ?s ?sname ?p ?pname 
WHERE { 
    ?s ?p wd:Q15891399 .
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q523039'), ('sname', 'André Theuriet'), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]


1

In [13]:
# André Theuriet Q523039

queryString = """
SELECT ?s ?sname ?p ?pname 
WHERE { 
    wd:Q523039 ?s ?p .
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)


Results
[('s', 'http://www.wikidata.org/prop/direct/P103'), ('sname', 'native language'), ('p', 'http://www.wikidata.org/entity/Q150'), ('pname', 'French')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q49757'), ('pname', 'poet')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q6625963'), ('pname', 'novelist')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q214917'), ('pname', 'playwright')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q36180'), ('pname', 'writer')]
[('s', 'http://www.wikidata.org/prop/direct/P119'), ('sname', 'place of burial'), ('p', 'http://www.wikidata.org/entity/Q42554963'), ('pname', 'Cimetière de Bourg-la-Reine')]
[('s', 'http://www.wikidata.org/prop/direct/P1343'), ('sname', 'described by 

32

In [14]:
# André Theuriet Q523039

queryString = """
SELECT ?s ?sname ?p ?pname 
WHERE { 
    ?s ?p wd:Q523039 .
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q15891399'), ('sname', 'Category:André Theuriet'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q70495940'), ('sname', 'Vice Chair of the French Academy'), ('p', 'http://www.wikidata.org/prop/direct/P1308'), ('pname', 'officeholder')]
[('s', 'http://www.wikidata.org/entity/Q3446959'), ('sname', 'rue André-Theuriet'), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('s', 'http://www.wikidata.org/entity/Q17360541'), ('sname', 'Odes et Chansons'), ('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]
[('s', 'http://www.wikidata.org/entity/Q17360543'), ('sname', 'Portrait'), ('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]
[('s', 'http://www.wikidata.org/entity/Q17360544'), ('sname', 'La Ferme au fond des bois'), ('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]
[('s', 'http:

19

In [15]:
# André Theuriet Q523039
# showing the books of this writer
queryString = """
SELECT ?s ?sname 
WHERE { 
    ?s wdt:P50 wd:Q523039 .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17360541'), ('sname', 'Odes et Chansons')]
[('s', 'http://www.wikidata.org/entity/Q17360543'), ('sname', 'Portrait')]
[('s', 'http://www.wikidata.org/entity/Q17360544'), ('sname', 'La Ferme au fond des bois')]
[('s', 'http://www.wikidata.org/entity/Q17360545'), ('sname', 'Horrida Bella')]
[('s', 'http://www.wikidata.org/entity/Q102067529'), ('sname', 'Le mariage de Gerard')]
[('s', 'http://www.wikidata.org/entity/Q105266480'), ('sname', 'Le mariage de Gerard')]
[('s', 'http://www.wikidata.org/entity/Q101111403'), ('sname', "Amour d'automne")]
[('s', 'http://www.wikidata.org/entity/Q105104167'), ('sname', "Amour d'automne")]
[('s', 'http://www.wikidata.org/entity/Q17360542'), ('sname', 'L’Alouette')]
[('s', 'http://www.wikidata.org/entity/Q102045895'), ('sname', 'Lucile Desenclos: une ondine')]
[('s', 'http://www.wikidata.org/entity/Q105267909'), ('sname', 'Lucile Desenclos: une ondine')]
[('s', 'http://www.wikidata.org/entity/Q105266511')

13

In [16]:
# date
# Le mariage de Gerard Q105266480

queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q105266480 ?s ?p .
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 

"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P123'), ('sname', 'publisher'), ('p', 'http://www.wikidata.org/entity/Q3117417'), ('pname', 'Groupe Flammarion')]
[('s', 'http://www.wikidata.org/prop/direct/P291'), ('sname', 'place of publication'), ('p', 'http://www.wikidata.org/entity/Q90'), ('pname', 'Paris')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q3331189'), ('pname', 'version, edition, or translation')]
[('s', 'http://www.wikidata.org/prop/direct/P407'), ('sname', 'language of work or name'), ('p', 'http://www.wikidata.org/entity/Q150'), ('pname', 'French')]
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q523039'), ('pname', 'André Theuriet')]


5

In [17]:
### searching for it's publication date
# Le mariage de Gerard Q105266480
# version, edition, or translation Q3331189
queryString = """
SELECT *
WHERE { 
    wd:Q3331189 ?p ?o .
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
} 

"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('o', 'http://www.wikidata.org/entity/Q16388'), ('oname', 'Functional Requirements for Bibliographic Records'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1382'), ('o', 'http://www.wikidata.org/entity/Q15994174'), ('oname', 'Fassung'), ('pname', 'partially coincident with')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('o', 'http://www.wikidata.org/entity/Q5858283'), ('oname', 'Template:Infobox book'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('o', 'http://www.wikidata.org/entity/P629'), ('oname', 'edition or translation of'), ('pname', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('o', 'http://www.wikidata.org/entity/P747'), ('oname', 'has edition or translation'), ('pname', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('o', 'http://www.wikidata.org/entity/Q571'), ('oname', 'book'), ('

38

In [18]:
# Le mariage de Gerard Q105266480
# version, edition, or translation Q3331189
### searching for it's publication date
queryString = """
SELECT *
WHERE { 
    wd:Q3331189 ?p ?o .
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
} 

"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('o', 'http://www.wikidata.org/entity/Q16388'), ('oname', 'Functional Requirements for Bibliographic Records'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1382'), ('o', 'http://www.wikidata.org/entity/Q15994174'), ('oname', 'Fassung'), ('pname', 'partially coincident with')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('o', 'http://www.wikidata.org/entity/Q5858283'), ('oname', 'Template:Infobox book'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('o', 'http://www.wikidata.org/entity/P629'), ('oname', 'edition or translation of'), ('pname', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('o', 'http://www.wikidata.org/entity/P747'), ('oname', 'has edition or translation'), ('pname', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('o', 'http://www.wikidata.org/entity/Q571'), ('oname', 'book'), ('

38

In [19]:
# publication date P577
# Le mariage de Gerard Q105266480
# version, edition, or translation Q3331189
### searching for it's publication date
queryString = """
SELECT ?p ?pname
WHERE { 
    wd:Q3331189 ?p wd:P577 .
    ?p <http://schema.org/name> ?pname .
} 

"""
print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type')]


1

In [20]:
#L’Alouette Q17360542
# publication date P577
# Odes et Chansons Q17360541
### searching for it's publication date
queryString = """
SELECT ?time
WHERE { 
    wd:Q17360542 wdt:P577 ?time .
} 

"""
print("Results")
run_query(queryString)

Results
[('time', '1866-01-01T00:00:00Z')]


1

In [21]:
# all italian and french writers:
queryString = """
SELECT ?frenchwritername ?italianwritername 
WHERE { 
    
    ?frenchwriter wdt:P27 wd:Q142 .
    ?italianwriter wdt:P27 wd:Q38 ;
                   wdt:P27 wd:Q172579 .
    
    ?frenchwriter wdt:P106 wd:Q36180 .
    ?italianwriter wdt:P106 wd:Q36180 .
    
    #labels
    ?frenchwriter <http://schema.org/name> ?frenchwritername .
    ?italianwriter <http://schema.org/name> ?italianwritername .

    
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('frenchwritername', 'Valery Larbaud'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Renat Lavaud'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Georges Lafenestre'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'René Doumic'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Maurice Dreyfous'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Téodor de Wyzewa'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'André Theuriet'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Philippe Chatrier'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Robert de Montesquiou'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Bonaventure des Périers'), ('italianwritername', 'Philip Kwok')]


10

In [22]:
# all italian and french people
queryString = """
SELECT ?writer ?writername 
WHERE { 
    
    ?writer wdt:P27 wd:Q142 .
    ?writer wdt:P27 wd:Q38 .
    ?writer wdt:P27 wd:Q172579 .

    
    #labels
    ?writer <http://schema.org/name> ?writername .
    
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('writer', 'http://www.wikidata.org/entity/Q503355'), ('writername', 'Igor Markevitch')]
[('writer', 'http://www.wikidata.org/entity/Q271885'), ('writername', 'Marisa Pavan')]
[('writer', 'http://www.wikidata.org/entity/Q543779'), ('writername', 'Marcello Pagliero')]
[('writer', 'http://www.wikidata.org/entity/Q2008943'), ('writername', 'Romeo Bosetti')]
[('writer', 'http://www.wikidata.org/entity/Q60322266'), ('writername', 'Alvaro Giorgetti')]
[('writer', 'http://www.wikidata.org/entity/Q28131352'), ('writername', 'Francis Rigon')]
[('writer', 'http://www.wikidata.org/entity/Q1917800'), ('writername', 'Pierre Brambilla')]
[('writer', 'http://www.wikidata.org/entity/Q206244'), ('writername', 'Stéphane Grappelli')]
[('writer', 'http://www.wikidata.org/entity/Q3266383'), ('writername', 'Luigi Barral')]
[('writer', 'http://www.wikidata.org/entity/Q10263113'), ('writername', 'César Garin')]


10

In [23]:
# all italian and french writer
queryString = """
SELECT ?writer ?writername 
WHERE { 
    
    ?writer wdt:P27 wd:Q142 .
    ?writer wdt:P27 wd:Q38 .
    ?writer wdt:P27 wd:Q172579 ;
            wdt:P106 wd:Q36180 .

    #labels
    ?writer <http://schema.org/name> ?writername .
    
} 

"""
print("Results")
run_query(queryString)
# it doesnt show all people

Results
[('writer', 'http://www.wikidata.org/entity/Q3175072'), ('writername', 'Jean Vodaine')]
[('writer', 'http://www.wikidata.org/entity/Q554949'), ('writername', 'Lanza del Vasto')]
[('writer', 'http://www.wikidata.org/entity/Q3616889'), ('writername', 'Angelo Bertino Poli')]
[('writer', 'http://www.wikidata.org/entity/Q3107932'), ('writername', 'Giulio Cesare Silvagni')]
[('writer', 'http://www.wikidata.org/entity/Q2036029'), ('writername', 'Albert Spaggiari')]
[('writer', 'http://www.wikidata.org/entity/Q3622676'), ('writername', 'Aristide Merolli')]
[('writer', 'http://www.wikidata.org/entity/Q3839289'), ('writername', 'Luigi Aurigemma')]
[('writer', 'http://www.wikidata.org/entity/Q3796496'), ('writername', 'Ilo De Franceschi')]


8

In [24]:
# all italian and french writers:
queryString = """
SELECT ?frenchwritername ?italianwritername 
WHERE { 
    
    ?frenchwriter wdt:P27 wd:Q142 .
    ?italianwriter wdt:P27 wd:Q38 ;
                   wdt:P27 wd:Q172579 .
    
    ?frenchwriter wdt:P106 wd:Q36180 .
    ?italianwriter wdt:P106 wd:Q36180 .
    
    #labels
    ?frenchwriter <http://schema.org/name> ?frenchwritername .
    ?italianwriter <http://schema.org/name> ?italianwritername .

    
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('frenchwritername', 'Valery Larbaud'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Renat Lavaud'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Georges Lafenestre'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'René Doumic'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Maurice Dreyfous'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Téodor de Wyzewa'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'André Theuriet'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Philippe Chatrier'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Robert de Montesquiou'), ('italianwritername', 'Philip Kwok')]
[('frenchwritername', 'Bonaventure des Périers'), ('italianwritername', 'Philip Kwok')]


10

# Identify the BGP for obtaining the Italian and French writers who published a book in the last 50 years with the name of their books

In [25]:
## all books by french authors in last 50 years
queryString = """
SELECT ?bookname ?writername ?time
WHERE { 
    
    ?writer wdt:P27 wd:Q142 .
    ?writer wdt:P106 wd:Q36180 .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('bookname', 'The End of Eddy'), ('writername', 'Édouard Louis'), ('time', '2014-01-01T00:00:00Z')]
[('bookname', 'History of violence'), ('writername', 'Édouard Louis'), ('time', '2016-01-01T00:00:00Z')]
[('bookname', 'Who Killed My Father'), ('writername', 'Édouard Louis'), ('time', '2018-01-01T00:00:00Z')]
[('bookname', 'Je viens du sud'), ('writername', 'Pierre Delanoë'), ('time', '1981-01-01T00:00:00Z')]
[('bookname', 'The Search for Ancient China'), ('writername', 'Corinne Debaine-Francort'), ('time', '1998-01-01T00:00:00Z')]
[('bookname', 'For Your Own Good'), ('writername', 'Alice Miller'), ('time', '1980-01-01T00:00:00Z')]
[('bookname', 'Ports of Call'), ('writername', 'Amin Maalouf'), ('time', '1996-01-01T00:00:00Z')]
[('bookname', 'Asterix and the Great Divide'), ('writername', 'Albert Uderzo'), ('time', '1980-01-01T00:00:00Z')]
[('bookname', 'Asterix and the Black Gold'), ('writername', 'Albert Uderzo'), ('time', '1981-01-01T00:00:00Z')]
[('bookname', 'Obelix and C

20

In [26]:
## all books by french authors in last 50 years
queryString = """
SELECT ?writername (GROUP_CONCAT(DISTINCT ?bookname; SEPARATOR=", ") AS ?booknames) 
WHERE { 
    
    ?writer wdt:P27 wd:Q142 .
    ?writer wdt:P106 wd:Q36180 .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('writername', 'Anne Calife'), ('booknames', "Conte d'Asphalte, Fleur de peau, La déferlante, Paul et le chat")]
[('writername', 'Max Gallo'), ('booknames', 'Victor Hugo')]
[('writername', 'Mathias Malzieu'), ('booknames', 'La Mécanique du cœur')]
[('writername', 'Jean-François Samlong'), ('booknames', 'Un soleil en exil')]
[('writername', 'Geneviève Huriet'), ('booknames', "Family Moving Day, Periwinkle at the Full Moon Ball, Violette's Daring Adventure")]
[('writername', 'Thérèse Moreau'), ('booknames', 'feminization dictionary')]
[('writername', 'Alex Capus'), ('booknames', 'A Matter of Time')]
[('writername', 'Émile Zola'), ('booknames', 'My hatreds ; Mes Haines')]
[('writername', 'Régine Deforges'), ('booknames', '101, avenue Henri-Martin, La Bicyclette bleue')]
[('writername', 'Édouard-Raoul Brygoo'), ('booknames', 'Notes sur les Brookesia de Madagascar IX. Observations sur B. tuberculata Mocquard, 1894, B. ramanantsoai sp. nov. et B. peyrierasi nom. nov. (Reptilia, Squa

20

# The number of books written in the last 50 years by French writers

In [27]:
#the number of books written in the last 50 years by French writers

queryString = """
 SELECT (COUNT (DISTINCT ?bookname) AS ?countbook) WHERE
              {
 
        ?writer wdt:P27 wd:Q142 .
        ?writer wdt:P106 wd:Q36180 .
    
        ?book wdt:P50 ?writer .
        ?book wdt:P577 ?time .
        FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
        #labels
        ?book <http://schema.org/name> ?bookname .
        ?writer <http://schema.org/name> ?writername .
               }
        
"""

print("Results")
run_query(queryString)

Results
[('countbook', '1734')]


1

In [28]:
#the number of books written in the last 50 years by French writers
queryString = """
 SELECT ?writername ?writer (COUNT (DISTINCT ?bookname) AS ?countbook) WHERE
              {
 
        ?writer wdt:P27 wd:Q142 .
        ?writer wdt:P106 wd:Q36180 .
    
        ?book wdt:P50 ?writer .
        ?book wdt:P577 ?time .
        FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
        #labels
        ?book <http://schema.org/name> ?bookname .
        ?writer <http://schema.org/name> ?writername .
               }
        GROUP BY ?writername ?writer
        ORDER BY DESC(?countbook) 
        LIMIT 35
"""

print("Results")
run_query(queryString)

Results
[('writername', 'Jean Weissenbach'), ('writer', 'http://www.wikidata.org/entity/Q1685910'), ('countbook', '192')]
[('writername', 'Stéphane Velut'), ('writer', 'http://www.wikidata.org/entity/Q41840536'), ('countbook', '53')]
[('writername', 'Pierre Bourdieu'), ('writer', 'http://www.wikidata.org/entity/Q156268'), ('countbook', '44')]
[('writername', 'Jean-Marie Gustave Le Clézio'), ('writer', 'http://www.wikidata.org/entity/Q42037'), ('countbook', '41')]
[('writername', 'Barbara Demeneix'), ('writer', 'http://www.wikidata.org/entity/Q30323562'), ('countbook', '31')]
[('writername', 'Gilles Deleuze'), ('writer', 'http://www.wikidata.org/entity/Q184226'), ('countbook', '27')]
[('writername', 'Milan Kundera'), ('writer', 'http://www.wikidata.org/entity/Q93166'), ('countbook', '21')]
[('writername', 'Thomas Piketty'), ('writer', 'http://www.wikidata.org/entity/Q984448'), ('countbook', '21')]
[('writername', 'Bernard Werber'), ('writer', 'http://www.wikidata.org/entity/Q12054'), ('

35

### The number of books written in the last 50 years by French writers

In [29]:
# the number of books written in the last 50 years by French writers
queryString = """
 SELECT ?writername (COUNT (DISTINCT ?bookname) AS ?countbook) WHERE
              {
 
        ?writer wdt:P27 wd:Q142 .
        ?writer wdt:P106 wd:Q36180 .
    
        ?book wdt:P50 ?writer .
        ?book wdt:P577 ?time .
        FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
        #labels
        ?book <http://schema.org/name> ?bookname .
        ?writer <http://schema.org/name> ?writername .
               }
        GROUP BY ?writername ?writer
        ORDER BY DESC(?countbook) 
        LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('writername', 'Jean Weissenbach'), ('countbook', '192')]
[('writername', 'Stéphane Velut'), ('countbook', '53')]
[('writername', 'Pierre Bourdieu'), ('countbook', '44')]
[('writername', 'Jean-Marie Gustave Le Clézio'), ('countbook', '41')]
[('writername', 'Barbara Demeneix'), ('countbook', '31')]
[('writername', 'Gilles Deleuze'), ('countbook', '27')]
[('writername', 'Milan Kundera'), ('countbook', '21')]
[('writername', 'Thomas Piketty'), ('countbook', '21')]
[('writername', 'Bernard Werber'), ('countbook', '18')]
[('writername', 'Gérald Tenenbaum'), ('countbook', '16')]


10

In [30]:
#wd:Q1685910 had the most book written
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
   wd:Q1685910 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P166'), ('sname', 'award received'), ('p', 'http://www.wikidata.org/entity/Q3320352'), ('pname', 'Princess of Asturias Award for Technical and Scientific Research')]
[('s', 'http://www.wikidata.org/prop/direct/P166'), ('sname', 'award received'), ('p', 'http://www.wikidata.org/entity/Q10855271'), ('pname', 'Knight of the Legion of Honour')]
[('s', 'http://www.wikidata.org/prop/direct/P27'), ('sname', 'country of citizenship'), ('p', 'http://www.wikidata.org/entity/Q142'), ('pname', 'France')]
[('s', 'http://www.wikidata.org/prop/direct/P1412'), ('sname', 'languages spoken, written or signed'), ('p', 'http://www.wikidata.org/entity/Q150'), ('pname', 'French')]
[('s', 'http://www.wikidata.org/prop/direct/P463'), ('sname', 'member of'), ('p', 'http://www.wikidata.org/entity/Q188771'), ('pname', 'French Academy of Sciences')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity

16

### All books by Italian authors in last 50 years

In [31]:
## all books by italian authors in last 50 years
queryString = """
SELECT ?italianwritername (GROUP_CONCAT(DISTINCT ?bookname; SEPARATOR=", ") AS ?booknames)
WHERE { 

    {?italianwriter wdt:P27 wd:Q38 }
    OPTIONAL { ?italianwriter wdt:P27 wd:Q172579 } 
    ?italianwriter wdt:P106 wd:Q36180 .

    
    ?book wdt:P50 ?italianwriter .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?italianwriter <http://schema.org/name> ?italianwritername .
} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('italianwritername', 'Anna Banti'), ('booknames', 'Un grido lacerante')]
[('italianwritername', 'Magdalena Tulli'), ('booknames', 'Dreams and Stones')]
[('italianwritername', 'Nicola Nosengo'), ('booknames', "365 days: Nature's 10, Appeals judge frees earthquake scientists accused of manslaughter, Biotechnology at the bar, Can artificial intelligence create the next wonder material?, Can you teach old drugs new tricks?, DNA heist that wasn’t sparks data questions, European deal aims to attract Arab scientists, Fertilized to death, Gran Sasso: Chamber of physics, Italian court finds seismologists guilty of manslaughter, Italian scientists fight tightened rules on animal testing, Italian seismologists cleared of manslaughter, Italian seismologists fight to overturn convictions, Italian universities get report cards, Italy puts seismology in the dock., L'Aquila verdict row grows, Marine biology network launches into choppy waters., Neutrinos make a splash in Italy., Piglets add 

20

In [32]:
# count of french writers

queryString = """
SELECT (COUNT (DISTINCT ?frenchwriter) AS ?countfrenchwriter) 
WHERE { 

    ?frenchwriter wdt:P27 wd:Q142 .
    ?frenchwriter wdt:P106 wd:Q36180 .
} 

"""
print("Results")
run_query(queryString)


Results
[('countfrenchwriter', '14400')]


1

In [33]:
## all Kingdom of Italy  authors
queryString = """
SELECT (COUNT (DISTINCT ?italianwriter) AS ?countitalianwriter) 
WHERE { 

    ?italianwriter wdt:P27 wd:Q172579 .
    ?italianwriter wdt:P106 wd:Q36180 .
} 

"""
print("Results")
run_query(queryString)


Results
[('countitalianwriter', '3436')]


1

In [34]:
## all italy authors 
queryString = """
SELECT (COUNT (DISTINCT ?italianwriter) AS ?countitalianwriter) 
WHERE { 

    ?italianwriter wdt:P27 wd:Q38 .
    ?italianwriter wdt:P106 wd:Q36180 .
} 

"""
print("Results")
run_query(queryString)


Results
[('countitalianwriter', '6264')]


1

In [35]:
## italian authors 
queryString = """
SELECT (COUNT (DISTINCT ?italianwriter) AS ?countitalianwriter) 
WHERE { 
    
    {?italianwriter wdt:P27 wd:Q38 }
    OPTIONAL { ?italianwriter wdt:P27 wd:Q172579 } 
    ?italianwriter wdt:P106 wd:Q36180 .

} 

"""
print("Results")
run_query(queryString)


Results
[('countitalianwriter', '6264')]


1

In [36]:
#all italy + Kingdom of Italy authors in last 50 years
queryString = """
SELECT (COUNT (DISTINCT ?italianwriter) AS ?countitalianwriter) 
WHERE { 
    
    {?italianwriter wdt:P27 wd:Q38 }
    UNION { ?italianwriter wdt:P27 wd:Q172579 } 
    ?italianwriter wdt:P106 wd:Q36180 .

} 

"""
print("Results")
run_query(queryString)


Results
[('countitalianwriter', '7442')]


1

In [37]:
## +Kingdom of Italy+italy +france authors
queryString = """
SELECT (COUNT (DISTINCT ?writer) AS ?countwriter) 
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
} 

"""
print("Results")
run_query(queryString)    

Results
[('countwriter', '21821')]


1

In [38]:
## all books by italian authors in last 50 years with their country of citizen
queryString = """
SELECT ?bookname ?italianwritername ?citizenname ?time
WHERE { 
    {?italianwriter wdt:P27 wd:Q38 }
    UNION { ?italianwriter wdt:P27 wd:Q172579 } 
    ?italianwriter wdt:P106 wd:Q36180 .
    ?italianwriter wdt:P27 ?citizen .
    
    ?book wdt:P50 ?italianwriter .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?italianwriter <http://schema.org/name> ?italianwritername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 20
"""
print("Results")
run_query(queryString)


Results
[('bookname', "L'amore è un dio"), ('italianwritername', 'Eva Cantarella'), ('citizenname', 'Italy'), ('time', '2007-01-01T00:00:00Z')]
[('bookname', 'Bisexuality in the ancient Word'), ('italianwritername', 'Eva Cantarella'), ('citizenname', 'Italy'), ('time', '1988-01-01T00:00:00Z')]
[('bookname', 'Indian Summer'), ('italianwritername', 'Milo Manara'), ('citizenname', 'Italy'), ('time', '1983-01-01T00:00:00Z')]
[('bookname', 'Gomorrah'), ('italianwritername', 'Roberto Saviano'), ('citizenname', 'Italy'), ('time', '2006-01-01T00:00:00Z')]
[('bookname', 'Those Who Leave and Those Who Stay'), ('italianwritername', 'Elena Ferrante'), ('citizenname', 'Italy'), ('time', '2013-01-01T00:00:00Z')]
[('bookname', 'Sotto il sole giaguaro'), ('italianwritername', 'Italo Calvino'), ('citizenname', 'Kingdom of Italy'), ('time', '1986-01-01T00:00:00Z')]
[('bookname', 'Sotto il sole giaguaro'), ('italianwritername', 'Italo Calvino'), ('citizenname', 'Italy'), ('time', '1986-01-01T00:00:00Z')]

20

### All books written by authors in last 50 years with their country of citizen that are Kingdom of Italy, Italy and France 

In [39]:
## all books by +Kingdom of Italy+italy +france authors in last 50 years with their country of citizen
queryString = """
SELECT ?bookname ?writername ?citizenname ?time
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 10
"""
print("Results")
run_query(queryString)


Results
[('bookname', "L'amore è un dio"), ('writername', 'Eva Cantarella'), ('citizenname', 'Italy'), ('time', '2007-01-01T00:00:00Z')]
[('bookname', 'Bisexuality in the ancient Word'), ('writername', 'Eva Cantarella'), ('citizenname', 'Italy'), ('time', '1988-01-01T00:00:00Z')]
[('bookname', 'Indian Summer'), ('writername', 'Milo Manara'), ('citizenname', 'Italy'), ('time', '1983-01-01T00:00:00Z')]
[('bookname', 'Gomorrah'), ('writername', 'Roberto Saviano'), ('citizenname', 'Italy'), ('time', '2006-01-01T00:00:00Z')]
[('bookname', 'Those Who Leave and Those Who Stay'), ('writername', 'Elena Ferrante'), ('citizenname', 'Italy'), ('time', '2013-01-01T00:00:00Z')]
[('bookname', 'Sotto il sole giaguaro'), ('writername', 'Italo Calvino'), ('citizenname', 'Kingdom of Italy'), ('time', '1986-01-01T00:00:00Z')]
[('bookname', 'Sotto il sole giaguaro'), ('writername', 'Italo Calvino'), ('citizenname', 'Italy'), ('time', '1986-01-01T00:00:00Z')]
[('bookname', 'Zero Zero Zero'), ('writername', 

10

In [40]:
## all books by +Kingdom of Italy+italy +france authors in last 50 years with their country of citizen
queryString = """
SELECT ?writername (GROUP_CONCAT(DISTINCT ?bookname; SEPARATOR=", ") AS ?booknames) ?citizenname
WHERE {  
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 10
"""
print("Results")
run_query(queryString)


Results
[('writername', 'Curzio Malaparte'), ('booknames', 'Maledetti toscani, Tecnica del colpo di Stato'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Jacques Priol'), ('booknames', 'Ne laissez pas Google gérer nos villes !'), ('citizenname', 'France')]
[('writername', 'Claude Moliterni'), ('booknames', 'Les aventures de la BD'), ('citizenname', 'France')]
[('writername', 'Giorgio Montefoschi'), ('booknames', 'House of the father'), ('citizenname', 'Italy')]
[('writername', 'Silvia Federici'), ('booknames', 'Caliban and the Witch'), ('citizenname', 'Italy')]
[('writername', 'Victor Hugo'), ('booknames', 'The Man Who Laughs, [Budget. 1848].'), ('citizenname', 'France')]
[('writername', 'Cyril Gély'), ('booknames', 'Fabrika'), ('citizenname', 'France')]
[('writername', 'Norberto Bobbio'), ('booknames', 'Legal Positivism'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Igor Škamperle'), ('booknames', 'Kraljeva hči'), ('citizenname', 'Italy')]
[('writername', 'Giorgio A

10

## 1) Identify the BGP for obtaining the Italian and French writers who published a book in the last 50 years

In [41]:
## all Kingdom of Italy+italy +france authors in last 50 years with their country of citizen
queryString = """
SELECT DISTINCT ?writername ?citizenname 
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('writername', 'Italo Calvino'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Natalia Ginzburg'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Maria Antonietta Macciocchi'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Alberto Bevilacqua'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Cesarina Vighy'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Mario Soldati'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Renzo De Felice'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Sergio Tofano'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Lino Aldani'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Michelangelo Antonioni'), ('citizenname', 'Kingdom of Italy')]
[('writername', 'Roberto Pazzi'), ('citizenname', 'Italy')]
[('writername', 'Eva Cantarella'), ('citizenname', 'Italy')]
[('writername', 'Milo Manara'), ('citizenname', 'Italy')]
[('writername', 'Roberto Saviano'), ('citizenname', 'Italy')]
[('writer

20

In [42]:
## all Kingdom of Italy+italy +france authors in last 50 years with their country of citizen
queryString = """
SELECT DISTINCT ?writer ?writername ?citizenname 
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('writer', 'http://www.wikidata.org/entity/Q154756'), ('writername', 'Italo Calvino'), ('citizenname', 'Kingdom of Italy')]
[('writer', 'http://www.wikidata.org/entity/Q275985'), ('writername', 'Natalia Ginzburg'), ('citizenname', 'Kingdom of Italy')]
[('writer', 'http://www.wikidata.org/entity/Q1083986'), ('writername', 'Maria Antonietta Macciocchi'), ('citizenname', 'Kingdom of Italy')]
[('writer', 'http://www.wikidata.org/entity/Q1130258'), ('writername', 'Alberto Bevilacqua'), ('citizenname', 'Kingdom of Italy')]
[('writer', 'http://www.wikidata.org/entity/Q599424'), ('writername', 'Cesarina Vighy'), ('citizenname', 'Kingdom of Italy')]
[('writer', 'http://www.wikidata.org/entity/Q59210'), ('writername', 'Mario Soldati'), ('citizenname', 'Kingdom of Italy')]
[('writer', 'http://www.wikidata.org/entity/Q438273'), ('writername', 'Renzo De Felice'), ('citizenname', 'Kingdom of Italy')]
[('writer', 'http://www.wikidata.org/entity/Q982381'), ('writername', 'Sergio Tofano'), ('c

50

In [43]:
# there is even people who have two citizenship, for instance: 'Alberto Moravia'
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    ?s ?p wd:Q244234 .
    #label
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q115134'), ('sname', 'Aida Turturro'), ('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pname', 'relative')]
[('s', 'http://www.wikidata.org/entity/Q3404523'), ('sname', 'David di Donatello for Best Foreign Actor'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q530923'), ('sname', 'Primetime Emmy Award for Outstanding Guest Actor in a Comedy Series'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q651722'), ('sname', 'Quiz Show'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q15614650'), ('sname', "God's Pocket"), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q130142'), ('sname', "Miller's Crossing"), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cas

50

In [44]:
# there is even people who have two citizenship, for instance: 'Alberto Moravia'
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q244234 ?s ?p .
    #label
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P800'), ('sname', 'notable work'), ('p', 'http://www.wikidata.org/entity/Q651722'), ('pname', 'Quiz Show')]
[('s', 'http://www.wikidata.org/prop/direct/P734'), ('sname', 'family name'), ('p', 'http://www.wikidata.org/entity/Q37536925'), ('pname', 'Turturro')]
[('s', 'http://www.wikidata.org/prop/direct/P69'), ('sname', 'educated at'), ('p', 'http://www.wikidata.org/entity/Q1140241'), ('pname', 'State University of New York')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q10798782'), ('pname', 'television actor')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q10800557'), ('pname', 'film actor')]
[('s', 'http://www.wikidata.org/prop/direct/P166'), ('sname', 'award received'), ('p', 'http://www.wikidata.org/entity/Q1131356'), ('pname', 'Theatre World Award')]
[('s', 'http://www.wikidata.org/prop/dire

39

In [45]:
queryString = """
SELECT ?p ?pname
WHERE { 
    wd:Q244234 wdt:P27 ?p .
    
    #label
    
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q30'), ('pname', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q38'), ('pname', 'Italy')]


2

## 2) Compare the number of books written by Italian and French writers

## the number of books written by Italian writers

In [46]:
# the number of books written by Italian writers

queryString = """

SELECT (COUNT (DISTINCT ?book) AS ?countitalianbook)
WHERE { 
 
  { ?italianwriter wdt:P27 wd:Q38 }
  UNION
  { ?italianwriter wdt:P27 wd:Q172579 }  

    ?italianwriter wdt:P106 wd:Q36180 .

    
    ?book wdt:P50 ?italianwriter .
    #labels
    
    #?italianwriter <http://schema.org/name> ?italianwritername .
} 

"""

print("Results")
run_query(queryString)


Results
[('countitalianbook', '5771')]


1

## the number of books written by French writers

In [47]:
# the number of books written by French writers

queryString = """

SELECT (COUNT (DISTINCT ?book) AS ?countbook)
WHERE { 

    ?frenchwriter wdt:P27 wd:Q142 .
    ?frenchwriter wdt:P106 wd:Q36180 .
    ?book wdt:P50 ?frenchwriter .
} 

"""

print("Results")
run_query(queryString)

Results
[('countbook', '38455')]


1

In [48]:
# all italian and french writers:
queryString = """
SELECT (COUNT (DISTINCT ?writer) AS ?countwriter)
WHERE { 
    {?writer wdt:P27 wd:Q142 } 
    UNION {?writer wdt:P27 wd:Q38 } 
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    #labels
    ?writer <http://schema.org/name> ?writername .

    
} 

"""
print("Results")
run_query(queryString)

Results
[('countwriter', '21774')]


1

In [49]:
## all books by +Kingdom of Italy+italy +france authors with their country of citizen
queryString = """
SELECT ?bookname ?writername ?citizenname 
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)


Results
[('bookname', "L'amore è un dio"), ('writername', 'Eva Cantarella'), ('citizenname', 'Italy')]
[('bookname', 'Bisexuality in the ancient Word'), ('writername', 'Eva Cantarella'), ('citizenname', 'Italy')]
[('bookname', 'Indian Summer'), ('writername', 'Milo Manara'), ('citizenname', 'Italy')]
[('bookname', 'Click'), ('writername', 'Milo Manara'), ('citizenname', 'Italy')]
[('bookname', 'The Garden of the Finzi-Continis'), ('writername', 'Giorgio Bassani'), ('citizenname', 'Kingdom of Italy')]
[('bookname', 'The Garden of the Finzi-Continis'), ('writername', 'Giorgio Bassani'), ('citizenname', 'Italy')]
[('bookname', 'Conversations in Sicily'), ('writername', 'Elio Vittorini'), ('citizenname', 'Kingdom of Italy')]
[('bookname', 'Conversations in Sicily'), ('writername', 'Elio Vittorini'), ('citizenname', 'Italy')]
[('bookname', 'The Castle of Crossed Destinies'), ('writername', 'Italo Calvino'), ('citizenname', 'Kingdom of Italy')]
[('bookname', 'The Castle of Crossed Destinies'

50

In [50]:
## ALL BOOKS from authors with France and Italy (+kingdom of Italy) nationality.
queryString = """
SELECT (COUNT (DISTINCT ?book) AS ?countbook)
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
"""
print("Results")
run_query(queryString)


Results
[('countbook', '9299')]


1

# 3) Count how many books written by Italian authors are now released with a "public domain" copyright form

In [51]:
## all books by +Kingdom of Italy+italy authors with their country of citizen
queryString = """
SELECT ?book ?bookname ?writer ?writername ?citizenname 
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q17635217'), ('bookname', "L'amore è un dio"), ('writer', 'http://www.wikidata.org/entity/Q459650'), ('writername', 'Eva Cantarella'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q17652232'), ('bookname', 'Bisexuality in the ancient Word'), ('writer', 'http://www.wikidata.org/entity/Q459650'), ('writername', 'Eva Cantarella'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q18745523'), ('bookname', 'Indian Summer'), ('writer', 'http://www.wikidata.org/entity/Q505666'), ('writername', 'Milo Manara'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q2915264'), ('bookname', 'Click'), ('writer', 'http://www.wikidata.org/entity/Q505666'), ('writername', 'Milo Manara'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q533391'), ('bookname', 'The Garden of the Finzi-Continis'), ('writer', 'http://www.wikidata.org/entity/Q318309'), ('writername', 'Giorgio Bassa

50

In [52]:

# [('book', 'http://www.wikidata.org/entity/Q18745523'), 
#[('bookname', 'Indian Summer'), ('writer', 'http://www.wikidata.org/entity/Q505666'),
#('writername', 'Milo Manara'), ('citizenname', 'Italy')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q18745523 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q505666'), ('pname', 'Milo Manara')]
[('s', 'http://www.wikidata.org/prop/direct/P495'), ('sname', 'country of origin'), ('p', 'http://www.wikidata.org/entity/Q38'), ('pname', 'Italy')]
[('s', 'http://www.wikidata.org/prop/direct/P407'), ('sname', 'language of work or name'), ('p', 'http://www.wikidata.org/entity/Q652'), ('pname', 'Italian')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q2831984'), ('pname', 'comic book album')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q1004'), ('pname', 'comics')]
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q12113'), ('pname', 'Hugo Pratt')]
[('s', 'http://www.wikidata.org/prop/direct/P1433'), ('sname', 'published in'), ('p', 'ht

7

In [53]:

# [('book', 'http://www.wikidata.org/entity/Q18745523'), 
#[('bookname', 'Indian Summer'), ('writer', 'http://www.wikidata.org/entity/Q505666'),
#('writername', 'Milo Manara'), ('citizenname', 'Italy')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q17652232 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q459650'), ('pname', 'Eva Cantarella')]
[('s', 'http://www.wikidata.org/prop/direct/P136'), ('sname', 'genre'), ('p', 'http://www.wikidata.org/entity/Q35760'), ('pname', 'essay')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q47461344'), ('pname', 'written work')]
[('s', 'http://www.wikidata.org/prop/direct/P407'), ('sname', 'language of work or name'), ('p', 'http://www.wikidata.org/entity/Q652'), ('pname', 'Italian')]
[('s', 'http://www.wikidata.org/prop/direct/P921'), ('sname', 'main subject'), ('p', 'http://www.wikidata.org/entity/Q21201'), ('pname', 'sociology')]


5

In [54]:
## all Kingdom of Italy+italy authors in last 2 years with their country of citizen
queryString = """
SELECT DISTINCT ?book ?writer ?writername ?citizenname 
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("2020-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q104477534'), ('writer', 'http://www.wikidata.org/entity/Q3779177'), ('writername', 'Guido Barbujani'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q104501070'), ('writer', 'http://www.wikidata.org/entity/Q3779177'), ('writername', 'Guido Barbujani'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q98202308'), ('writer', 'http://www.wikidata.org/entity/Q3903499'), ('writername', 'Piersandro Pallavicini'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q96020300'), ('writer', 'http://www.wikidata.org/entity/Q3903499'), ('writername', 'Piersandro Pallavicini'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q104745095'), ('writer', 'http://www.wikidata.org/entity/Q1042470'), ('writername', 'Carlo Rovelli'), ('citizenname', 'United States of America')]
[('book', 'http://www.wikidata.org/entity/Q104745095'), ('writer', 'http://www.wikidata.org/entity/Q104

17

In [55]:
#[('book', 'http://www.wikidata.org/entity/Q104477534'), 
#('writer', 'http://www.wikidata.org/entity/Q3779177'), ('writername', 'Guido Barbujani'), ('citizenname', 'Italy')]

queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q104477534 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q13442814'), ('pname', 'scholarly article')]
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q3779177'), ('pname', 'Guido Barbujani')]
[('s', 'http://www.wikidata.org/prop/direct/P1433'), ('sname', 'published in'), ('p', 'http://www.wikidata.org/entity/Q5532699'), ('pname', 'Genes')]
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q104477531'), ('pname', 'Andrea Ceolin')]
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q104477532'), ('pname', 'Cristina Guardiano')]


5

In [56]:
#[('book', 'http://www.wikidata.org/entity/Q104477534'), 
#('writer', 'http://www.wikidata.org/entity/Q3779177'), ('writername', 'Guido Barbujani'), ('citizenname', 'Italy')]

queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q3779177 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q3126128'), ('pname', 'geneticist')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q36180'), ('pname', 'writer')]
[('s', 'http://www.wikidata.org/prop/direct/P27'), ('sname', 'country of citizenship'), ('p', 'http://www.wikidata.org/entity/Q38'), ('pname', 'Italy')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q5'), ('pname', 'human')]
[('s', 'http://www.wikidata.org/prop/direct/P1343'), ('sname', 'described by source'), ('p', 'http://www.wikidata.org/entity/Q51955019'), ('pname', 'BEIC Digital Library')]
[('s', 'http://www.wikidata.org/prop/direct/P735'), ('sname', 'given name'), ('p', 'http://www.wikidata.org/entity/Q639762'), ('pname', 'Guido')]
[('s', 'http://www.wikidata.org/prop/direct/P1412'), ('sname', 'languages spoken,

14

In [57]:
# [('s', 'http://www.wikidata.org/prop/direct/P1343'), ('sname', 'described by source'),
#('p', 'http://www.wikidata.org/entity/Q51955019'), ('pname', 'BEIC Digital Library')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q51955019 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q35127'), ('pname', 'website')]
[('s', 'http://www.wikidata.org/prop/direct/P17'), ('sname', 'country'), ('p', 'http://www.wikidata.org/entity/Q38'), ('pname', 'Italy')]
[('s', 'http://www.wikidata.org/prop/direct/P131'), ('sname', 'located in the administrative territorial entity'), ('p', 'http://www.wikidata.org/entity/Q490'), ('pname', 'Milan')]
[('s', 'http://www.wikidata.org/prop/direct/P5008'), ('sname', 'on focus list of Wikimedia project'), ('p', 'http://www.wikidata.org/entity/Q106447762'), ('pname', 'Wikidata:Gruppo Wikidata per Musei, Archivi e Biblioteche/Riviste italiane di biblioteconomia')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q2352616'), ('pname', 'catalogue')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entit

8

In [58]:
## all Kingdom of Italy+italy authors between 1810 until 1910 with their country of citizen
queryString = """
SELECT DISTINCT ?book ?writer ?writername ?citizenname 
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1810-01-01"^^xsd:dateTime <= ?time && ?time < "1910-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q19621681'), ('writer', 'http://www.wikidata.org/entity/Q529858'), ('writername', 'Giovanni Papini'), ('citizenname', 'Kingdom of Italy')]
[('book', 'http://www.wikidata.org/entity/Q106729263'), ('writer', 'http://www.wikidata.org/entity/Q529858'), ('writername', 'Giovanni Papini'), ('citizenname', 'Kingdom of Italy')]
[('book', 'http://www.wikidata.org/entity/Q19621681'), ('writer', 'http://www.wikidata.org/entity/Q529858'), ('writername', 'Giovanni Papini'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q106729263'), ('writer', 'http://www.wikidata.org/entity/Q529858'), ('writername', 'Giovanni Papini'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q106729263'), ('writer', 'http://www.wikidata.org/entity/Q1227513'), ('writername', 'Giuseppe Prezzolini'), ('citizenname', 'Kingdom of Italy')]
[('book', 'http://www.wikidata.org/entity/Q106729263'), ('writer', 'http://www.wikidata.org/entity/Q1

50

In [59]:
#[('book', 'http://www.wikidata.org/entity/Q19621681'),
#('writer', 'http://www.wikidata.org/entity/Q529858'), ('writername', 'Giovanni Papini'), ('citizenname', 'Kingdom of Italy')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q19621681 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q529858'), ('pname', 'Giovanni Papini')]
[('s', 'http://www.wikidata.org/prop/direct/P6216'), ('sname', 'copyright status'), ('p', 'http://www.wikidata.org/entity/Q19652'), ('pname', 'public domain')]
[('s', 'http://www.wikidata.org/prop/direct/P136'), ('sname', 'genre'), ('p', 'http://www.wikidata.org/entity/Q35760'), ('pname', 'essay')]
[('s', 'http://www.wikidata.org/prop/direct/P495'), ('sname', 'country of origin'), ('p', 'http://www.wikidata.org/entity/Q38'), ('pname', 'Italy')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q47461344'), ('pname', 'written work')]
[('s', 'http://www.wikidata.org/prop/direct/P921'), ('sname', 'main subject'), ('p', 'http://www.wikidata.org/entity/Q5891'), ('pname', 'philosophy')]
[('s', 'http://www.wikidata.org/prop/direct/P407'), ('sname', 'language of work or nam

7

In [60]:
queryString = """
SELECT ?book ?bookname
WHERE { 
    ?book wdt:P6216 wd:Q19652 .
    #labels
    ?book <http://schema.org/name> ?bookname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q8187529'), ('bookname', 'A Studio at Les Batignolles')]
[('book', 'http://www.wikidata.org/entity/Q1218985'), ('bookname', 'The Last Days of Pompeii')]
[('book', 'http://www.wikidata.org/entity/Q20158334'), ('bookname', 'Aristanax 2 (Pauly-Wissowa)')]
[('book', 'http://www.wikidata.org/entity/Q216299'), ('bookname', 'Suda')]
[('book', 'http://www.wikidata.org/entity/Q15584559'), ('bookname', 'Salvation Nell')]
[('book', 'http://www.wikidata.org/entity/Q794123'), ('bookname', 'The Siege of Beszterce')]
[('book', 'http://www.wikidata.org/entity/Q12360731'), ('bookname', 'The Phantom of the Opera')]
[('book', 'http://www.wikidata.org/entity/Q15726350'), ('bookname', 'Ae fond kiss, and then we sever...')]
[('book', 'http://www.wikidata.org/entity/Q15726373'), ('bookname', 'Why should we idly waste our prime...')]
[('book', 'http://www.wikidata.org/entity/Q15727864'), ('bookname', 'The Dance Lesson')]
[('book', 'http://www.wikidata.org/enti

50

In [61]:
# all books by Kingdom of Italy+italy authors between 1810 until 1910 with their country of citizen + their copyright status.

#[('s', 'http://www.wikidata.org/prop/direct/P6216'),
#('sname', 'copyright status'),
#('p', 'http://www.wikidata.org/entity/Q19652'), ('pname', 'public domain')]

queryString = """
SELECT DISTINCT ?bookname ?writername
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P6216 wd:Q19652 .
    ?book wdt:P577 ?time .
    FILTER("1810-01-01"^^xsd:dateTime <= ?time && ?time < "1910-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .

} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('bookname', 'Il Crepuscolo dei Filosofi'), ('writername', 'Giovanni Papini')]
[('bookname', 'The Tavern Knight'), ('writername', 'Rafael Sabatini')]
[('bookname', 'Una donna'), ('writername', 'Sibilla Aleramo')]
[('bookname', "'O Sole Mio"), ('writername', 'Giovanni Capurro')]
[('bookname', 'Ashes'), ('writername', 'Grazia Deledda')]
[('bookname', 'Elias Portolu'), ('writername', 'Grazia Deledda')]
[('bookname', 'Il trionfo della morte'), ('writername', "Gabriele D'Annunzio")]
[('bookname', 'The Intruder'), ('writername', "Gabriele D'Annunzio")]
[('bookname', 'Le vergini delle rocce'), ('writername', "Gabriele D'Annunzio")]
[('bookname', 'Primo vere'), ('writername', "Gabriele D'Annunzio")]
[('bookname', 'Heart'), ('writername', 'Edmondo de Amicis')]
[('bookname', 'Beatrice Cenci'), ('writername', 'Francesco Domenico Guerrazzi')]
[('bookname', 'Giovanni Episcopo'), ('writername', "Gabriele D'Annunzio")]
[('bookname', 'The Fire'), ('writername', "Gabriele D'Annunzio")]
[('book

50

# 3) Count how many books written by Italian authors are now released with a "public domain" copyright form

In [62]:
queryString = """
SELECT (COUNT (DISTINCT ?book) AS ?countbook)
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?book wdt:P50 ?writer .
    ?book wdt:P6216 wd:Q19652 .
    #labels
    ?book <http://schema.org/name> ?bookname .

} 

"""
print("Results")
run_query(queryString)

Results
[('countbook', '176')]


1

## 4) How many Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 

In [63]:
# 'Alberto Moravia'
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q244234 ?s ?p .
    #label
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P800'), ('sname', 'notable work'), ('p', 'http://www.wikidata.org/entity/Q651722'), ('pname', 'Quiz Show')]
[('s', 'http://www.wikidata.org/prop/direct/P734'), ('sname', 'family name'), ('p', 'http://www.wikidata.org/entity/Q37536925'), ('pname', 'Turturro')]
[('s', 'http://www.wikidata.org/prop/direct/P69'), ('sname', 'educated at'), ('p', 'http://www.wikidata.org/entity/Q1140241'), ('pname', 'State University of New York')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q10798782'), ('pname', 'television actor')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q10800557'), ('pname', 'film actor')]
[('s', 'http://www.wikidata.org/prop/direct/P166'), ('sname', 'award received'), ('p', 'http://www.wikidata.org/entity/Q1131356'), ('pname', 'Theatre World Award')]
[('s', 'http://www.wikidata.org/prop/dire

39

In [64]:
#[('s', 'http://www.wikidata.org/prop/direct/P166'), ('sname', 'award received'), 
#('p', 'http://www.wikidata.org/entity/Q1131356'), ('pname', 'Theatre World Award')]
# 'Alberto Moravia'
queryString = """
SELECT DISTINCT ?award ?awardname
WHERE { 
    wd:Q244234 wdt:P166 ?award .
    #label
    ?s <http://schema.org/name> ?sname .
    ?award <http://schema.org/name> ?awardname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1044427'), ('awardname', 'Primetime Emmy Award')]
[('award', 'http://www.wikidata.org/entity/Q42369'), ('awardname', 'Cannes Film Festival')]
[('award', 'http://www.wikidata.org/entity/Q1131356'), ('awardname', 'Theatre World Award')]
[('award', 'http://www.wikidata.org/entity/Q530923'), ('awardname', 'Primetime Emmy Award for Outstanding Guest Actor in a Comedy Series')]


4

In [65]:
# books with public domain and their awards
queryString = """
SELECT ?writername ?awardname ?award
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?book wdt:P50 ?writer .
    ?book wdt:P6216 wd:Q19652 .
    ?writer wdt:P166 ?award .
    #labels
    ?writer <http://schema.org/name> ?writername .
    ?award <http://schema.org/name> ?awardname .

} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('writername', 'Natalia Ginzburg'), ('awardname', 'Charles Veillon prize in the Italian language'), ('award', 'http://www.wikidata.org/entity/Q23061993')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'Bagutta Prize'), ('award', 'http://www.wikidata.org/entity/Q788568')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'Strega Prize'), ('award', 'http://www.wikidata.org/entity/Q731542')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'AAAS Fellow'), ('award', 'http://www.wikidata.org/entity/Q52382875')]
[('writername', 'Giovanni Papini'), ('awardname', 'Marzotto Prize'), ('award', 'http://www.wikidata.org/entity/Q3910584')]
[('writername', 'Giovanni Papini'), ('awardname', 'Marzotto Prize'), ('award', 'http://www.wikidata.org/entity/Q3910584')]
[('writername', 'Giovanni Papini'), ('awardname', 'Marzotto Prize'), ('award', 'http://www.wikidata.org/entity/Q3910584')]
[('writername', 'Giovanni Papini'), ('awardname', 'Marzotto Prize'), ('award', 'http://www.wikidata.or

50

In [66]:
#award: Q23061993
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q23061993 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)
#?award wdt:P31 wd:Q378427 .

Results
[('s', 'http://www.wikidata.org/prop/direct/P361'), ('sname', 'part of'), ('p', 'http://www.wikidata.org/entity/Q16671101'), ('pname', 'Charles Veillon prize')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q378427'), ('pname', 'literary award')]


2

In [67]:
# 'Alberto Moravia'
queryString = """
SELECT ?award ?awardname
WHERE { 
    wd:Q244234 wdt:P166 ?award .
    ?award wdt:P31 wd:Q378427 .
    #label
    ?s <http://schema.org/name> ?sname .
    ?award <http://schema.org/name> ?awardname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
Empty


0

In [68]:
queryString = """
SELECT ?writername ?awardname ?award
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .
    ?award wdt:P31 wd:Q378427 .
    #labels
    ?writer <http://schema.org/name> ?writername .
    ?award <http://schema.org/name> ?awardname .

} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('writername', 'Eva Cantarella'), ('awardname', 'Bagutta Prize'), ('award', 'http://www.wikidata.org/entity/Q788568')]
[('writername', 'Eva Cantarella'), ('awardname', 'Bagutta Prize'), ('award', 'http://www.wikidata.org/entity/Q788568')]
[('writername', 'Eva Cantarella'), ('awardname', 'Bagutta Prize'), ('award', 'http://www.wikidata.org/entity/Q788568')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'Strega Prize'), ('award', 'http://www.wikidata.org/entity/Q731542')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'Strega Prize'), ('award', 'http://www.wikidata.org/entity/Q731542')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'Strega Prize'), ('award', 'http://www.wikidata.org/entity/Q731542')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'Strega Prize'), ('award', 'http://www.wikidata.org/entity/Q731542')]
[('writername', 'Natalia Ginzburg'), ('awardname', 'Strega Prize'), ('award', 'http://www.wikidata.org/entity/Q731542')]
[('writername', 'Natalia Gi

50

In [69]:
queryString = """
SELECT (COUNT (DISTINCT ?award) AS ?countaward)
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    #?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .
    # ?award wdt:P31 wd:Q378427 .
    #### this is a Literary award
    #labels
    ?award <http://schema.org/name> ?awardname .
} 

"""
print("Results")
run_query(queryString)

Results
[('countaward', '467')]


1

In [70]:
queryString = """
SELECT ?award ?awardname
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    #?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .
    # ?award wdt:P31 wd:Q378427 .
    #### this is a Literary award
    #labels
    ?award <http://schema.org/name> ?awardname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q23061993'), ('awardname', 'Charles Veillon prize in the Italian language')]
[('award', 'http://www.wikidata.org/entity/Q23061993'), ('awardname', 'Charles Veillon prize in the Italian language')]
[('award', 'http://www.wikidata.org/entity/Q2108706'), ('awardname', 'Premio Campiello')]
[('award', 'http://www.wikidata.org/entity/Q2108706'), ('awardname', 'Premio Campiello')]
[('award', 'http://www.wikidata.org/entity/Q118404'), ('awardname', 'Geschwister-Scholl-Preis')]
[('award', 'http://www.wikidata.org/entity/Q519214'), ('awardname', 'Nelly Sachs Prize')]
[('award', 'http://www.wikidata.org/entity/Q7650000'), ('awardname', 'Sutherland Trophy')]
[('award', 'http://www.wikidata.org/entity/Q94578019'), ('awardname', 'Serena Medal')]
[('award', 'http://www.wikidata.org/entity/Q1158951'), ('awardname', 'Dan David Prize')]
[('award', 'http://www.wikidata.org/entity/Q1526935'), ('awardname', 'Viareggio Prize')]
[('award', 'http://www.wikida

50

In [71]:
# How many Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 
queryString = """
SELECT (COUNT (?book) AS ?bookaward)
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 

"""
print("Results")
run_query(queryString)

Results
[('bookaward', '120')]


1

# How many Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 

### The Literature Nobel winners from Italy and from the Kingdom of Italy.

In [72]:
# The Literature Nobel winners from Italy and from the Kingdom of Italy.
queryString = """
SELECT DISTINCT ?writername ?awardname ?bookname ?citizenname
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 
LIMIT 20
"""
print("Results")
run_query(queryString)


Results
[('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature'), ('bookname', 'He Had Two Pistols with White and Black Eyes'), ('citizenname', 'Italy')]
[('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature'), ('bookname', 'Mamma Togni'), ('citizenname', 'Italy')]
[('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature'), ('bookname', 'The Devil with Boobs'), ('citizenname', 'Italy')]
[('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature'), ('bookname', 'The Open Couple'), ('citizenname', 'Italy')]
[('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature'), ('bookname', 'Accidental Death of an Anarchist'), ('citizenname', 'Italy')]
[('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature'), ('bookname', 'Elizabeth: Almost by Chance a Woman'), ('citizenname', 'Italy')]
[('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature'), ('bookname', 'Trumpets and Raspberries'), ('citizenname', 'I

20

In [73]:
# The Literature Nobel winners from Italy and from the Kingdom of Italy.
queryString = """
SELECT ?writername ?citizenname (GROUP_CONCAT(DISTINCT ?bookname; SEPARATOR=", ") AS ?booknames)
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 
LIMIT 10
"""
print("Results")
run_query(queryString)


Results
[('writername', 'Grazia Deledda'), ('citizenname', 'Kingdom of Italy'), ('booknames', "After the divorce, Annalena Bilsini, Ashes, Canne al vento, Elias Portolu, L'Edera, Po rozwodzie, Popiół, The Mother")]
[('writername', 'Luigi Pirandello'), ('citizenname', 'Kingdom of Italy'), ('booknames', "A Day Goes By, As You Desire Me, As you desire me, At the Exit, Bellavita, Berecche and the War, Cecè, Chwe chymeriad yn chwilio am awdur, Cinci, Clothing The Naked, Clothing the Naked, Clothing the naked, Così è, Così è (se vi pare), Crow Discovers the Moon, Each In His Own Way, Enrico IV, First Night, From the Nose to the Sky, Gea's Easter, Henri IV, Henry IV, Her Husband, Il piacere dell'onestà, In the Whirpool, It Is so  (if you think so), It is so, if you think so, L'Esclusa, La Rallegrata, La disdetta di Pitagora, La favola del figlio cambiato, Liola, Ma non è una cosa seria, Naked, Nené and Ninì, No one knows how, Novels for a year, One, No one and One Hundred Thousand, Questa ser

4

In [74]:
# How many Literature Nobel awards won authors from Italy? 
queryString = """
SELECT (COUNT (DISTINCT ?book) AS ?bookaward)
WHERE { 
    ?writer wdt:P27 wd:Q38 .
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 

"""
print("Results")
run_query(queryString)

Results
[('bookaward', '18')]


1

In [75]:
# How many Literature Nobel awards won authors from the Kingdom of Italy? 
queryString = """
SELECT (COUNT (DISTINCT ?book) AS ?countbook)
WHERE { 
    ?writer wdt:P27 wd:Q172579 .
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 

"""
print("Results")
run_query(queryString)

Results
[('countbook', '102')]


1

## Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [76]:
# How many Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 
### the award is literary #^
queryString = """
SELECT DISTINCT ?bookname ?book ?writername ?awardname
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .
    ?book wdt:P50 ?writer .
    ?award wdt:P31 wd:Q378427 .
    #### this is a Literary award
    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    #?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 
LIMIT 10

"""
print("Results")
run_query(queryString)

Results
[('bookname', 'Corpse for Sale'), ('book', 'http://www.wikidata.org/entity/Q5172601'), ('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature')]
[('bookname', 'Mistero Buffo'), ('book', 'http://www.wikidata.org/entity/Q1246586'), ('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature')]
[('bookname', 'The Open Couple'), ('book', 'http://www.wikidata.org/entity/Q1528183'), ('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature')]
[('bookname', "Archangels Don't Play Pinball"), ('book', 'http://www.wikidata.org/entity/Q3772074'), ('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature')]
[('bookname', 'Mamma Togni'), ('book', 'http://www.wikidata.org/entity/Q6745735'), ('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature')]
[('bookname', 'Johan Padan and the Discovery of the Americas'), ('book', 'http://www.wikidata.org/entity/Q3180437'), ('writername', 'Dario Fo'), ('awardname', 'Nobel Prize in Literature')]


10

In [77]:
# `wd:Q213678` Vatican Library    
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q213678 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P463'), ('sname', 'member of'), ('p', 'http://www.wikidata.org/entity/Q3487972'), ('pname', 'Société de l’histoire de France')]
[('s', 'http://www.wikidata.org/prop/direct/P131'), ('sname', 'located in the administrative territorial entity'), ('p', 'http://www.wikidata.org/entity/Q237'), ('pname', 'Vatican City')]
[('s', 'http://www.wikidata.org/prop/direct/P17'), ('sname', 'country'), ('p', 'http://www.wikidata.org/entity/Q237'), ('pname', 'Vatican City')]
[('s', 'http://www.wikidata.org/prop/direct/P1343'), ('sname', 'described by source'), ('p', 'http://www.wikidata.org/entity/Q602358'), ('pname', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q22806'), ('pname', 'national library')]
[('s', 'http://www.wikidata.org/prop/direct/P463'), ('sname', 'member of'), ('p', 'http://www.wikidata.org/entity/Q35677307'), ('pname',

11

In [78]:
# `wd:Q213678`
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    ?s ?p wd:Q213678 .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q19823475'), ('sname', 'prefect of the Vatican Library'), ('p', 'http://www.wikidata.org/prop/direct/P2389'), ('pname', 'organization directed by the office or position')]
[('s', 'http://www.wikidata.org/entity/Q2754987'), ('sname', 'Leopoldo Cicognara'), ('p', 'http://www.wikidata.org/prop/direct/P9419'), ('pname', 'personal library at')]
[('s', 'http://www.wikidata.org/entity/Q711620'), ('sname', 'Stefano Evodio Assemani'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q3606806'), ('sname', 'Agostino Paravicini Bagliani'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q1230408'), ('sname', 'Gugliemo Sirleto'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q21545450'), ('sname', 'Pietro Matranga'), ('p', 'http://www.wikidata.org/prop/direct/

50

In [79]:
# `wd:Q213678`
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    ?s ?p wd:Q213678 .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
    FILTER regex(?pname, "collection", "i") 
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17174857'), ('sname', 'Bibbia di Federico da Montefeltro - BAV Urb.Lat.1&2'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q209285'), ('sname', 'Codex Vaticanus'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q1806588'), ('sname', 'Lasimos Krater'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q664820'), ('sname', 'Bodmer Papyri'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q1106341'), ('sname', 'Codex Vaticanus Graecus 64'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q3592714'), ('sname', 'Barberini Gospels'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s'

50

In [80]:
# `wd:Q213678`
queryString = """
SELECT DISTINCT ?s ?sname
WHERE { 
    ?s wdt:P276 wd:Q213678 .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2258615'), ('sname', 'Rossi Codex')]
[('s', 'http://www.wikidata.org/entity/Q77463433'), ('sname', 'Apostolic Library Reg. 1323')]
[('s', 'http://www.wikidata.org/entity/Q29366989'), ('sname', 'Missale Gallicanum vetus - BAV Pal.lat.493')]
[('s', 'http://www.wikidata.org/entity/Q1250195'), ('sname', 'Codex Assemanius')]
[('s', 'http://www.wikidata.org/entity/Q1180580'), ('sname', 'De arte venandi cum avibus')]
[('s', 'http://www.wikidata.org/entity/Q744655'), ('sname', 'Chigi codex')]
[('s', 'http://www.wikidata.org/entity/Q101438843'), ('sname', 'Rossiana Library')]
[('s', 'http://www.wikidata.org/entity/Q3656584'), ('sname', 'Canzoniere Vaticano latino 3793')]
[('s', 'http://www.wikidata.org/entity/Q16912485'), ('sname', 'Divine Comedy Illustrated by Botticelli')]
[('s', 'http://www.wikidata.org/entity/Q12274931'), ('sname', 'Vatican Palimpsest')]


10

In [81]:
# `wd:Q37922`
queryString = """
SELECT DISTINCT ?sname ?p ?pname
WHERE { 
    ?s ?p wd:Q37922 .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('sname', 'Valery Larbaud'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Adrien Albert Marie de Mun'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Alan Sillitoe'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Pierre Emmanuel'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Ramón Pérez de Ayala'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Franz Theodor Csokor'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Rudolf Alexander Schröder'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Giorgos Theotokás'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sname', 'Johannes V. Jensen'), ('p', 'http://www.wikidata.org/prop/dire

10

In [82]:
# `wd:Q37922`
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE { 
    wd:Q37922 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P1346'), ('sname', 'winner'), ('p', 'http://www.wikidata.org/entity/Q159552'), ('pname', 'Johannes V. Jensen')]
[('s', 'http://www.wikidata.org/prop/direct/P1346'), ('sname', 'winner'), ('p', 'http://www.wikidata.org/entity/Q254032'), ('pname', 'Olga Tokarczuk')]
[('s', 'http://www.wikidata.org/prop/direct/P1027'), ('sname', 'conferred by'), ('p', 'http://www.wikidata.org/entity/Q207360'), ('pname', 'Swedish Academy')]
[('s', 'http://www.wikidata.org/prop/direct/P138'), ('sname', 'named after'), ('p', 'http://www.wikidata.org/entity/Q23810'), ('pname', 'Alfred Nobel')]
[('s', 'http://www.wikidata.org/prop/direct/P279'), ('sname', 'subclass of'), ('p', 'http://www.wikidata.org/entity/Q7191'), ('pname', 'Nobel Prize')]
[('s', 'http://www.wikidata.org/prop/direct/P361'), ('sname', 'part of'), ('p', 'http://www.wikidata.org/entity/Q7191'), ('pname', 'Nobel Prize')]
[('s', 'http://www.wikidata.org/prop/direct/P1346'), ('sname', 'winner'), 

10

In [83]:
# `wd:Q213678`
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    ?s ?p wd:Q213678 .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
    FILTER regex(?pname, "collection", "i") 
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17174857'), ('sname', 'Bibbia di Federico da Montefeltro - BAV Urb.Lat.1&2'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q209285'), ('sname', 'Codex Vaticanus'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q1806588'), ('sname', 'Lasimos Krater'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q664820'), ('sname', 'Bodmer Papyri'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q1106341'), ('sname', 'Codex Vaticanus Graecus 64'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s', 'http://www.wikidata.org/entity/Q3592714'), ('sname', 'Barberini Gospels'), ('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('s'

50

In [84]:
#[('s', 'http://www.wikidata.org/entity/Q17174857'), 
#('sname', 'Bibbia di Federico da Montefeltro - BAV Urb.Lat.1&2'), 
#('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q17174857 ?s ?p .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)


Results
[('s', 'http://www.wikidata.org/prop/direct/P170'), ('sname', 'creator'), ('p', 'http://www.wikidata.org/entity/Q16185868'), ('pname', "Francesco d'Antonio del Chierico")]
[('s', 'http://www.wikidata.org/prop/direct/P195'), ('sname', 'collection'), ('p', 'http://www.wikidata.org/entity/Q213678'), ('pname', 'Vatican Library')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q48498'), ('pname', 'illuminated manuscript')]
[('s', 'http://www.wikidata.org/prop/direct/P88'), ('sname', 'commissioned by'), ('p', 'http://www.wikidata.org/entity/Q435501'), ('pname', 'Federico da Montefeltro')]


4

In [85]:
# the collection
queryString = """
SELECT ?p ?pname
WHERE { 
    wd:Q17174857 wdt:P195 ?p .
    #labels
    ?p <http://schema.org/name> ?pname .
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q213678'), ('pname', 'Vatican Library')]


1

In [86]:
# Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 
queryString = """
SELECT DISTINCT ?bookname ?book ?writername ?citizenname
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('bookname', 'He Had Two Pistols with White and Black Eyes'), ('book', 'http://www.wikidata.org/entity/Q3631052'), ('writername', 'Dario Fo'), ('citizenname', 'Italy')]
[('bookname', 'Mamma Togni'), ('book', 'http://www.wikidata.org/entity/Q6745735'), ('writername', 'Dario Fo'), ('citizenname', 'Italy')]
[('bookname', 'The Devil with Boobs'), ('book', 'http://www.wikidata.org/entity/Q7730023'), ('writername', 'Dario Fo'), ('citizenname', 'Italy')]
[('bookname', 'The Open Couple'), ('book', 'http://www.wikidata.org/entity/Q1528183'), ('writername', 'Dario Fo'), ('citizenname', 'Italy')]
[('bookname', 'Accidental Death of an Anarchist'), ('book', 'http://www.wikidata.org/entity/Q2704227'), ('writername', 'Dario Fo'), ('citizenname', 'Italy')]
[('bookname', 'Elizabeth: Almost by Chance a Woman'), ('book', 'http://www.wikidata.org/entity/Q5362235'), ('writername', 'Dario Fo'), ('citizenname', 'Italy')]
[('bookname', 'Trumpets and Raspberries'), ('book', 'http://www.wikidata.org/en

50

In [87]:
# [('bookname', 'Corpse for Sale'),
# ('book', 'http://www.wikidata.org/entity/Q5172601'),
# ('writername', 'Dario Fo'), ('citizenname', 'Italy')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q5172601 ?s ?p  .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q25379'), ('pname', 'play')]
[('s', 'http://www.wikidata.org/prop/direct/P50'), ('sname', 'author'), ('p', 'http://www.wikidata.org/entity/Q765'), ('pname', 'Dario Fo')]


2

In [88]:
# Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 
queryString = """
SELECT DISTINCT ?book ?writername ?writer ?citizenname
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q3631052'), ('writername', 'Dario Fo'), ('writer', 'http://www.wikidata.org/entity/Q765'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q6745735'), ('writername', 'Dario Fo'), ('writer', 'http://www.wikidata.org/entity/Q765'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q7730023'), ('writername', 'Dario Fo'), ('writer', 'http://www.wikidata.org/entity/Q765'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q1528183'), ('writername', 'Dario Fo'), ('writer', 'http://www.wikidata.org/entity/Q765'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q2704227'), ('writername', 'Dario Fo'), ('writer', 'http://www.wikidata.org/entity/Q765'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/entity/Q5362235'), ('writername', 'Dario Fo'), ('writer', 'http://www.wikidata.org/entity/Q765'), ('citizenname', 'Italy')]
[('book', 'http://www.wikidata.org/ent

50

In [89]:
#[('book', 'http://www.wikidata.org/entity/Q3795120'), 
# ('writername', 'Luigi Pirandello'), 
# ('writer', 'http://www.wikidata.org/entity/Q1403'), 
# ('citizenname', 'Kingdom of Italy')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q1403 ?s ?p  .
    #labels
    ?s <http://schema.org/name> ?sname .
    ?p <http://schema.org/name> ?pname .
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P509'), ('sname', 'cause of death'), ('p', 'http://www.wikidata.org/entity/Q12192'), ('pname', 'pneumonia')]
[('s', 'http://www.wikidata.org/prop/direct/P69'), ('sname', 'educated at'), ('p', 'http://www.wikidata.org/entity/Q152171'), ('pname', 'University of Bonn')]
[('s', 'http://www.wikidata.org/prop/direct/P27'), ('sname', 'country of citizenship'), ('p', 'http://www.wikidata.org/entity/Q172579'), ('pname', 'Kingdom of Italy')]
[('s', 'http://www.wikidata.org/prop/direct/P1343'), ('sname', 'described by source'), ('p', 'http://www.wikidata.org/entity/Q17378135'), ('pname', 'Great Soviet Encyclopedia (1969–1978)')]
[('s', 'http://www.wikidata.org/prop/direct/P1412'), ('sname', 'languages spoken, written or signed'), ('p', 'http://www.wikidata.org/entity/Q188'), ('pname', 'German')]
[('s', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation'), ('p', 'http://www.wikidata.org/entity/Q214917'), ('pname', 'playwright')]
[

41

In [90]:
# `wd:Q213678`  | Vatican Library        | node |
queryString = """
SELECT *
WHERE { 
    wd:Q213678 ?s ?p .
    #labels
    #?p <http://schema.org/name> ?pname .
    #?s <http://schema.org/name> ?sname .
    
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('p', 'http://wikiba.se/ontology#Item')]
[('s', 'http://www.wikidata.org/prop/direct/P5505'), ('p', '87743')]
[('s', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('p', 'https://livedata.bibsys.no/authority/90524924')]
[('s', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('p', 'http://viaf.org/viaf/146085455')]
[('s', 'http://www.wikidata.org/prop/direct-normalized/P2163'), ('p', 'http://id.worldcat.org/fast/543664')]
[('s', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('p', 'https://d-nb.info/gnd/1003302-6')]
[('s', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('p', 'https://id.loc.gov/authorities/names/n80050101')]
[('s', 'http://www.wikidata.org/prop/direct-normalized/P245'), ('p', 'http://vocab.getty.edu/ulan/500308607')]
[('s', 'http://www.wikidata.org/prop/direct-normalized/P2581'), ('p', 'http://babelnet.org/rdf/s03227550n')]
[('s', 'http://www.wikidata.org/prop/direct-n

10

In [91]:
#('p', 'http://www.wikidata.org/entity/Q8229359'), ('pname', 'Category:Vatican Library')]
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q8229359 ?s ?p .
    #labels
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P301'), ('sname', "category's main topic"), ('p', 'http://www.wikidata.org/entity/Q213678'), ('pname', 'Vatican Library')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sname', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q4167836'), ('pname', 'Wikimedia category')]
[('s', 'http://www.wikidata.org/prop/direct/P4224'), ('sname', 'category contains'), ('p', 'http://www.wikidata.org/entity/Q5'), ('pname', 'human')]


3

In [92]:
# `wd:Q213678`  | Vatican Library        | node |
queryString = """
SELECT COUNT(DISTINCT ?s)
WHERE { 
    wd:Q213678 ?s ?p .
    #labels
    #?p <http://schema.org/name> ?pname .
    #?s <http://schema.org/name> ?sname .
    
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('callret-0', '79')]


1

In [93]:
# `wd:Q213678`  | Vatican Library        | node |
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE { 
    wd:Q213678 ?s ?p .
    #labels
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P1037'), ('sname', 'director / manager'), ('p', 'http://www.wikidata.org/entity/Q4459693'), ('pname', 'José Tolentino de Mendonça')]
[('s', 'http://www.wikidata.org/prop/direct/P131'), ('sname', 'located in the administrative territorial entity'), ('p', 'http://www.wikidata.org/entity/Q237'), ('pname', 'Vatican City')]
[('s', 'http://www.wikidata.org/prop/direct/P1343'), ('sname', 'described by source'), ('p', 'http://www.wikidata.org/entity/Q602358'), ('pname', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('s', 'http://www.wikidata.org/prop/direct/P1687'), ('sname', 'Wikidata property'), ('p', 'http://www.wikidata.org/entity/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('s', 'http://www.wikidata.org/prop/direct/P17'), ('sname', 'country'), ('p', 'http://www.wikidata.org/entity/Q237'), ('pname', 'Vatican City')]
[('s', 'http://www.wikidata.org/prop/direct/P2388'), ('sname', 'office held by head of the organization'),

11

In [94]:
# `wd:Q213678`  | Vatican Library        | node |
queryString = """
SELECT COUNT (DISTINCT ?p)
WHERE { 
    ?s ?p wd:Q213678 .
    #labels
    #?p <http://schema.org/name> ?pname .
    #?s <http://schema.org/name> ?sname .
    
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('callret-0', '20')]


1

In [95]:
# `wd:Q213678`  | Vatican Library        | node |
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?s ?p wd:Q213678 .
    #labels
    ?p <http://schema.org/name> ?pname .
    #?s <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2389'), ('pname', 'organization directed by the office or position')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('pname', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pname', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('pname', 'affiliation')]
[('p', 'http://www.wikidata.org/prop/direct/P189'), ('pname', 'location of discovery')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P485'), ('pname', 'archives at')]
[('p',

18

In [96]:
#[('bookname', 'He Had Two Pistols with White and Black Eyes'), 
#('book', 'http://www.wikidata.org/entity/Q3631052'),
#('writername', 'Dario Fo'), ('citizenname', 'Italy')]
queryString = """
SELECT *
WHERE { 
    wd:Q3631052 ?p ?o .
    #labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2671'), ('o', 'http://g.co/kg/g/120m0_q_')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('o', '/g/120m0_q_')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q25379')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('o', 'http://www.wikidata.org/entity/Q652')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('o', 'http://www.wikidata.org/entity/Q38')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('o', 'http://www.wikidata.org/entity/Q765')]
[('p', 'http://www.wikidata.org/prop/direct/P577'), ('o', '1960-01-01T00:00:00Z')]
[('p', 'http://schema.org/name'), ('o', 'He Had Two Pistols with White and Black Eyes')]
[('p', 'http://www.wikidata.org/prop/direct/P1191'), ('o', '1960-09-02T00:00:00Z')]


10

In [97]:
#[('bookname', 'He Had Two Pistols with White and Black Eyes'), 
#('book', 'http://www.wikidata.org/entity/Q3631052'),
#('writername', 'Dario Fo'), ('citizenname', 'Italy')]
queryString = """
SELECT *
WHERE { 
    wd:Q3631052 ?p ?o .
    #labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('o', '/g/120m0_q_'), ('pname', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q25379'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('o', 'http://www.wikidata.org/entity/Q652'), ('pname', 'language of work or name')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('o', 'http://www.wikidata.org/entity/Q38'), ('pname', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('o', 'http://www.wikidata.org/entity/Q765'), ('pname', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P577'), ('o', '1960-01-01T00:00:00Z'), ('pname', 'publication date')]
[('p', 'http://www.wikidata.org/prop/direct/P1191'), ('o', '1960-09-02T00:00:00Z'), ('pname', 'date of first performance')]


7

In [98]:
#[('bookname', 'He Had Two Pistols with White and Black Eyes'), 
#('book', 'http://www.wikidata.org/entity/Q3631052'),
#('writername', 'Dario Fo'), ('citizenname', 'Italy')]
queryString = """
SELECT *
WHERE { 
    ?p ?o wd:Q3631052 .
    #labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('p', 'https://www.wikidata.org/wiki/Special:EntityData/Q3631052'), ('o', 'http://schema.org/about')]


1

In [99]:
# collection of a book.
# `wd:Q213678`  | Vatican Library        | node |
## [('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
## [('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
# [('p', 'http://www.wikidata.org/prop/direct/P485'), ('pname', 'archives at')]
queryString = """
SELECT *
WHERE { 
    ?s wdt:P195 wd:Q213678 .
    #?s wdt:P276 wd:Q213678 .
    #?s wdt:P361 wd:Q213678 .
    #?s wdt:P485 wd:Q213678 .
    
    #labels
    #?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17174857'), ('sname', 'Bibbia di Federico da Montefeltro - BAV Urb.Lat.1&2')]
[('s', 'http://www.wikidata.org/entity/Q209285'), ('sname', 'Codex Vaticanus')]
[('s', 'http://www.wikidata.org/entity/Q1806588'), ('sname', 'Lasimos Krater')]
[('s', 'http://www.wikidata.org/entity/Q664820'), ('sname', 'Bodmer Papyri')]
[('s', 'http://www.wikidata.org/entity/Q1106341'), ('sname', 'Codex Vaticanus Graecus 64')]
[('s', 'http://www.wikidata.org/entity/Q3592714'), ('sname', 'Barberini Gospels')]
[('s', 'http://www.wikidata.org/entity/Q16653651'), ('sname', 'Divina Commedia de Federigo da Montefeltro - BAVaticana UrbLat365')]
[('s', 'http://www.wikidata.org/entity/Q903320'), ('sname', 'Book of Dede Korkut')]
[('s', 'http://www.wikidata.org/entity/Q101438843'), ('sname', 'Rossiana Library')]
[('s', 'http://www.wikidata.org/entity/Q1531846'), ('sname', 'Vergilius Vaticanus')]
[('s', 'http://www.wikidata.org/entity/Q1937608'), ('sname', 'Minuscule 131'

25

In [100]:
# location of a book.
# `wd:Q213678`  | Vatican Library        | node |
## [('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
## [('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
# [('p', 'http://www.wikidata.org/prop/direct/P485'), ('pname', 'archives at')]
queryString = """
SELECT *
WHERE { 
    #?s wdt:P195 wd:Q213678 .
    ?s wdt:P276 wd:Q213678 .
    #?s wdt:P361 wd:Q213678 .
    #?s wdt:P485 wd:Q213678 .
    
    #labels
    #?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3592714'), ('sname', 'Barberini Gospels')]
[('s', 'http://www.wikidata.org/entity/Q101438843'), ('sname', 'Rossiana Library')]
[('s', 'http://www.wikidata.org/entity/Q338714'), ('sname', 'Anjou Legendarium')]
[('s', 'http://www.wikidata.org/entity/Q1250195'), ('sname', 'Codex Assemanius')]
[('s', 'http://www.wikidata.org/entity/Q1106235'), ('sname', 'Codex Ríos')]
[('s', 'http://www.wikidata.org/entity/Q33231619'), ('sname', 'breviary of  Matthias Corvin')]
[('s', 'http://www.wikidata.org/entity/Q77463414'), ('sname', 'Apostolic Library Regina 1682')]
[('s', 'http://www.wikidata.org/entity/Q77463433'), ('sname', 'Apostolic Library Reg. 1323')]
[('s', 'http://www.wikidata.org/entity/Q55831099'), ('sname', 'Codex vaticanus latinus 3199')]
[('s', 'http://www.wikidata.org/entity/Q16912485'), ('sname', 'Divine Comedy Illustrated by Botticelli')]
[('s', 'http://www.wikidata.org/entity/Q16939517'), ('sname', 'Codex Parisino-petropolitanus')]
[('

24

In [101]:
# a library of a book.

# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
# [('p', 'http://www.wikidata.org/prop/direct/P485'), ('pname', 'archives at')]
queryString = """
SELECT *
WHERE { 
    #?s wdt:P195 wd:Q213678 .
    #?s wdt:P276 wd:Q213678 .
    ?s wdt:P361 wd:Q213678 .
    #?s wdt:P485 wd:Q213678 .
    
    #labels
    #?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q101438843'), ('sname', 'Rossiana Library')]
[('s', 'http://www.wikidata.org/entity/Q105426233'), ('sname', 'Chart of the World (copy) Inv. 5R')]
[('s', 'http://www.wikidata.org/entity/Q105426133'), ('sname', 'Chart of the World (copy)')]


3

In [102]:
# [('p', 'http://www.wikidata.org/prop/direct/P485'), ('pname', 'archives at')]
queryString = """
SELECT *
WHERE { 
    #?s wdt:P195 wd:Q213678 .
    #?s wdt:P276 wd:Q213678 .
    #?s wdt:P361 wd:Q213678 .
    ?s wdt:P485 wd:Q213678 .
    
    #labels
    #?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
} 
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3057287'), ('sname', 'Ernst Steinmann')]


1

In [103]:
queryString = """
SELECT *
WHERE { 

   #wd:Q18223014 ?p ?o .
    wd:Q18223014 wdt:P577 ?publicationdate .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('publicationdate', '1904-01-01T00:00:00Z')]


1

In [104]:
# The Literature Nobel winners from Italy and from the Kingdom of Italy.
queryString = """
SELECT DISTINCT ?bookname ?book
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?book wdt:P50 ?writer .
    ?writer wdt:P166 ?award .
    ?book wdt:P577 ?publicationdate .

    #labels
    ?award <http://schema.org/name> ?awardname .
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name> ?citizenname .
    FILTER regex(?awardname, "nobel", "i") 
    
} 
ORDER BY (?publicationdate)
LIMIT 10
"""
print("Results")
run_query(queryString)


Results
[('bookname', 'To Satana'), ('book', 'http://www.wikidata.org/entity/Q15622813')]
[('bookname', 'Barbarian Odes'), ('book', 'http://www.wikidata.org/entity/Q3880861')]
[('bookname', 'Giosuè Carducci letter to Mario Rapisardi'), ('book', 'http://www.wikidata.org/entity/Q19143931')]
[('bookname', 'Giambi ed Epodi'), ('book', 'http://www.wikidata.org/entity/Q3762440')]
[('bookname', 'San Martino'), ('book', 'http://www.wikidata.org/entity/Q3947642')]
[('bookname', 'The Reality of the Dream'), ('book', 'http://www.wikidata.org/entity/Q48802313')]
[('bookname', 'Rime nuove'), ('book', 'http://www.wikidata.org/entity/Q3935849')]
[('bookname', "Gea's Easter"), ('book', 'http://www.wikidata.org/entity/Q26702469')]
[('bookname', 'The vice'), ('book', 'http://www.wikidata.org/entity/Q3823229')]
[('bookname', 'Elias Portolu'), ('book', 'http://www.wikidata.org/entity/Q2528617')]


10

In [105]:
queryString = """
SELECT *
WHERE { 

   wd:Q2528617 ?p ?o .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2671'), ('o', 'http://g.co/kg/g/1229k_r3')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('o', 'http://www.wikidata.org/entity/Q8261')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('o', 'topic/Elias-Portolu')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('o', 'http://www.wikidata.org/entity/Q4690699')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('o', 'http://www.wikidata.org/entity/Q18223014')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('o', '/g/1229k_r3')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q47461344')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('o', 'http://www.wikidata.org/entity/Q652')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('o', 'http://www.wikidata.org/entity/Q38')]
[('p', 'http://www.wikidata.org/p

17

In [106]:
queryString = """
SELECT *
WHERE { 

   ?p ?o wd:Q2528617 .
    # Labels
    
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q18223014'), ('o', 'http://www.wikidata.org/prop/direct/P155'), ('oname', 'follows')]
[('p', 'http://www.wikidata.org/entity/Q7728'), ('o', 'http://www.wikidata.org/prop/direct/P800'), ('oname', 'notable work')]
[('p', 'http://www.wikidata.org/entity/Q63439538'), ('o', 'http://www.wikidata.org/prop/direct/P629'), ('oname', 'edition or translation of')]


3

### Countries 

In [107]:
#| `wd:Q38`      | Italy         | node |
#| `wdt:P17`     | country       | predicate | 
queryString = """
SELECT COUNT (DISTINCT ?p)
WHERE { 

    ?p wdt:P17 wd:Q38 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "continent", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '267673')]


1

In [108]:
#| `wd:Q38`      | Italy         | node |
#| `wdt:P17`     | country       | predicate | 
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

    ?p wdt:P17 ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "iran", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1144408'), ('pname', 'House of Pico della Mirandola')]
[('p', 'http://www.wikidata.org/entity/Q3625430'), ('pname', 'Siege of Mirandola (1551)')]
[('p', 'http://www.wikidata.org/entity/Q3625432'), ('pname', 'Siege of Mirandola (1510)')]
[('p', 'http://www.wikidata.org/entity/Q42310167'), ('pname', 'Siege of Mirandola (1705)')]
[('p', 'http://www.wikidata.org/entity/Q42883809'), ('pname', 'Siege of Mirandola (1321)')]
[('p', 'http://www.wikidata.org/entity/Q42341887'), ('pname', 'Mirandola Mint')]
[('p', 'http://www.wikidata.org/entity/Q61717757'), ('pname', 'Mirandola witch trials')]
[('p', 'http://www.wikidata.org/entity/Q189326'), ('pname', 'Qajar dynasty')]
[('p', 'http://www.wikidata.org/entity/Q5718239'), ('pname', 'Qajar architecture')]
[('p', 'http://www.wikidata.org/entity/Q1072837'), ('pname', 'First Iranian national anthem')]
[('p', 'http://www.wikidata.org/entity/Q15267557'), ('pname', 'Ispahsalar')]
[('p', 'http://www.wikidata

20

In [109]:
#| `wd:Q5960201`      | Qajar Iran        | node |
#| `wdt:P17`     | country       | predicate | 
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

    wd:Q5960201 wdt:P17 ?o .
    # Labels
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?oname, "iran", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q63158027'), ('oname', 'Qajar Iran')]


1

In [110]:
#| `wd:Q5960201`      | Qajar Iran        | node |
#| `wdt:P17`     | country       | predicate | 
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

    wd:Q19568301 wdt:P17 ?o .
    # Labels
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?oname, "iran", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q794'), ('oname', 'Iran')]


1

In [111]:
## all Qajar authors. find where is their books now?
queryString = """
SELECT DISTINCT ?writername (GROUP_CONCAT(DISTINCT ?bookname; SEPARATOR=", ") AS ?booknames)
WHERE { 
    ?writer wdt:P27 wd:Q794 .
    ?writer wdt:P106 wd:Q36180 .
    #?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    #FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
} 
LIMIT 5
"""
print("Results")
run_query(queryString)

Results
[('writername', 'Mehdi Khalaji'), ('booknames', 'Natani')]
[('writername', 'Firoozeh Dumas'), ('booknames', 'Funny in Farsi')]
[('writername', 'Manijeh Razeghi'), ('booknames', 'Active and passive infrared imager based on short-wave and mid-wave type-II superlattice dual-band detectors, Advances in mid-infrared detection and imaging: a key issues review., Al(x)Ga(1-x)N-based deep-ultraviolet 320×256 focal plane array, Author Correction: Radiative recombination of confined electrons at the MgZnO/ZnO heterojunction interface., Bias-selectable nBn dual-band long-/very long-wavelength infrared photodetectors based on InAs/InAs1-xSbx/AlAs1-xSbx type-II superlattices., Bias-selectable three-color short-, extended-short-, and mid-wavelength infrared photodetectors based on type-II InAs/GaSb/AlSb superlattices., Dark current reduction in microjunction-based double electron barrier type-II InAs/InAsSb superlattice long-wavelength infrared photodetectors., Demonstration of type-II superl

5

In [112]:
## all Qajar authors. find where is their books now?
queryString = """
SELECT DISTINCT ?bookname
WHERE { 
    ?writer wdt:P27 wd:Q63158027 .
    ?writer wdt:P106 wd:Q36180 .
    #?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    #FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
} 
LIMIT 4
"""
print("Results")
run_query(queryString)

Results
[('bookname', 'Kitáb-i-Íqán')]
[('bookname', 'Akhbar Nameh')]
[('bookname', 'The Seven Valleys')]
[('bookname', 'Fire Tablet')]


4

In [113]:
#
queryString = """
SELECT DISTINCT ?writer ?writername ?book ?citizenname
WHERE { 
    ?writer wdt:P27 wd:Q794 .
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    #FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name>  ?citizenname .
} 
LIMIT 5
"""
print("Results")
run_query(queryString)

Results
[('writer', 'http://www.wikidata.org/entity/Q2298313'), ('writername', 'Reza Aslan'), ('book', 'http://www.wikidata.org/entity/Q16171363'), ('citizenname', 'Iran')]
[('writer', 'http://www.wikidata.org/entity/Q5959526'), ('writername', 'Mohammad Reza Sarshar'), ('book', 'http://www.wikidata.org/entity/Q17082322'), ('citizenname', 'Iran')]
[('writer', 'http://www.wikidata.org/entity/Q19629541'), ('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q33237001'), ('citizenname', 'Ireland')]
[('writer', 'http://www.wikidata.org/entity/Q19629541'), ('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q33237001'), ('citizenname', 'United States of America')]
[('writer', 'http://www.wikidata.org/entity/Q19629541'), ('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q33237001'), ('citizenname', 'Iran')]


5

In [114]:
# occupation
queryString = """
SELECT *
WHERE { 

    wd:Q19629541 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "occ", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

## wdt:P106

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q1622272'), ('pname', 'occupation'), ('oname', 'university teacher')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q169470'), ('pname', 'occupation'), ('oname', 'physicist')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q36180'), ('pname', 'occupation'), ('oname', 'writer')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q81096'), ('pname', 'occupation'), ('oname', 'engineer')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q901'), ('pname', 'occupation'), ('oname', 'scientist')]


5

In [115]:
## all Qajar authors. find where is their books now?
queryString = """
SELECT DISTINCT ?writer ?writername ?book
WHERE { 
    ?writer wdt:P27 wd:Q794 .
    ?writer wdt:P106 wd:Q36180 .
    #?writer wdt:P27 ?citizen .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    #FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
} 
LIMIT 5
"""
print("Results")
run_query(queryString)

Results
[('writer', 'http://www.wikidata.org/entity/Q2298313'), ('writername', 'Reza Aslan'), ('book', 'http://www.wikidata.org/entity/Q16171363')]
[('writer', 'http://www.wikidata.org/entity/Q5959526'), ('writername', 'Mohammad Reza Sarshar'), ('book', 'http://www.wikidata.org/entity/Q17082322')]
[('writer', 'http://www.wikidata.org/entity/Q19629541'), ('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q33237001')]
[('writer', 'http://www.wikidata.org/entity/Q803646'), ('writername', 'Bahram Bayzai'), ('book', 'http://www.wikidata.org/entity/Q20110094')]
[('writer', 'http://www.wikidata.org/entity/Q19629541'), ('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q28828505')]


5

In [116]:
# find the location of their books.
# show different citizenships of these writers and their occupation.
# [('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q28828505')]
# then find the location of their books.
queryString = """
SELECT *
WHERE { 

    wd:Q28828505 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P932'), ('o', '4897645'), ('pname', 'PMCID')]
[('p', 'http://www.wikidata.org/prop/direct/P304'), ('o', '25213'), ('pname', 'page(s)')]
[('p', 'http://www.wikidata.org/prop/direct/P478'), ('o', '6'), ('pname', 'volume')]
[('p', 'http://www.wikidata.org/prop/direct/P3181'), ('o', '940447'), ('pname', 'OpenCitations bibliographic resource ID')]
[('p', 'http://www.wikidata.org/prop/direct/P356'), ('o', '10.1038/SREP25213'), ('pname', 'DOI')]
[('p', 'http://www.wikidata.org/prop/direct/P2093'), ('o', 'Neelanjan Bandyopadhyay'), ('pname', 'author name string')]
[('p', 'http://www.wikidata.org/prop/direct/P2093'), ('o', 'Ryan McClintock'), ('pname', 'author name string')]
[('p', 'http://www.wikidata.org/prop/direct/P2860'), ('o', 'http://www.wikidata.org/entity/Q35629464'), ('pname', 'cites work')]
[('p', 'http://www.wikidata.org/prop/direct/P2860'), ('o', 'http://www.wikidata.org/entity/Q35773460'), ('pname', 'cites work')]
[('p', 'http://

20

In [117]:
# find the location of their books.
# show different citizenships of these writers and their occupation.
# [('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q28828505')]

queryString = """
SELECT COUNT(DISTINCT ?p)
WHERE { 

    wd:Q28828505 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('callret-0', '16')]


1

In [118]:
# show different citizenships of these writers and their occupation.
# [('writername', 'Manijeh Razeghi'), ('book', 'http://www.wikidata.org/entity/Q28828505')]
# show the place the book is published.
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

    wd:Q28828505 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P932'), ('pname', 'PMCID')]
[('p', 'http://www.wikidata.org/prop/direct/P304'), ('pname', 'page(s)')]
[('p', 'http://www.wikidata.org/prop/direct/P478'), ('pname', 'volume')]
[('p', 'http://www.wikidata.org/prop/direct/P3181'), ('pname', 'OpenCitations bibliographic resource ID')]
[('p', 'http://www.wikidata.org/prop/direct/P356'), ('pname', 'DOI')]
[('p', 'http://www.wikidata.org/prop/direct/P2093'), ('pname', 'author name string')]
[('p', 'http://www.wikidata.org/prop/direct/P2860'), ('pname', 'cites work')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P2888'), ('pname', 'exact match')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('pname', 'language of work or name')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]
[('p', 'http://www.wikidata.org/prop/dir

16

### Show different citizenships of Iranian writers and their occupation and count their citizenships.

In [119]:
queryString = """
SELECT DISTINCT ?writername 
(GROUP_CONCAT(DISTINCT ?bookname; SEPARATOR=", ") AS ?booknames)
(GROUP_CONCAT(DISTINCT ?occupationname; SEPARATOR=", ") AS ?occupationnames)
(GROUP_CONCAT(DISTINCT ?citizenname; SEPARATOR=", ") AS ?citizennames)
WHERE { 
    ?writer wdt:P27 wd:Q794 .
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?writer wdt:P106 ?occupation .
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    #FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?citizen <http://schema.org/name>  ?citizenname .
    ?occupation <http://schema.org/name> ?occupationname .
} 
LIMIT 5
"""
print("Results")
run_query(queryString)

Results
[('writername', 'Mehdi Khalaji'), ('booknames', 'Natani'), ('occupationnames', 'blogger, journalist, linguist, novelist, translator, writer'), ('citizennames', 'Iran')]
[('writername', 'Firoozeh Dumas'), ('booknames', 'Funny in Farsi'), ('occupationnames', 'biographer, writer'), ('citizennames', 'Iran, United States of America')]
[('writername', 'Manijeh Razeghi'), ('booknames', 'Active and passive infrared imager based on short-wave and mid-wave type-II superlattice dual-band detectors, Advances in mid-infrared detection and imaging: a key issues review., Al(x)Ga(1-x)N-based deep-ultraviolet 320×256 focal plane array, Author Correction: Radiative recombination of confined electrons at the MgZnO/ZnO heterojunction interface., Bias-selectable nBn dual-band long-/very long-wavelength infrared photodetectors based on InAs/InAs1-xSbx/AlAs1-xSbx type-II superlattices., Bias-selectable three-color short-, extended-short-, and mid-wavelength infrared photodetectors based on type-II In

5

### Show different citizenships of Italian and French writers and their occupation and count their citizenships.

In [120]:
queryString = """
SELECT ?writername 
(GROUP_CONCAT(DISTINCT ?bookname; SEPARATOR=", ") AS ?booknames)
(GROUP_CONCAT(DISTINCT ?occupationname; SEPARATOR=", ") AS ?occupationnames)
(GROUP_CONCAT(DISTINCT ?citizenname; SEPARATOR=", ") AS ?citizennames)
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION
    {?writer wdt:P27 wd:Q142 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    ?writer wdt:P27 ?citizen .
    ?writer wdt:P106 ?occupation .
    ?book wdt:P50 ?writer .
    ?book wdt:P577 ?time .
    FILTER("1972-01-01"^^xsd:dateTime <= ?time && ?time < "2022-01-01"^^xsd:dateTime).
    #labels
    ?book <http://schema.org/name> ?bookname .
    ?writer <http://schema.org/name> ?writername .
    ?citizen <http://schema.org/name> ?citizenname .
    ?occupation <http://schema.org/name> ?occupationname .
} 
ORDER BY (?time)
LIMIT 10
"""
print("Results")
run_query(queryString)


Results
[('writername', 'Romain Gary'), ('booknames', 'Europa'), ('occupationnames', 'French Resistance fighter, aircraft pilot, diplomat, film director, jurist, novelist, politician, science fiction writer, screenwriter, translator, writer'), ('citizennames', 'France, Poland, Russian Empire')]
[('writername', 'Simone de Beauvoir'), ('booknames', 'Adieux: A Farewell to Sartre'), ('occupationnames', 'author, autobiographer, diarist, essayist, journalist, literary critic, novelist, philosopher, political activist, political philosopher, women letter writer, writer'), ('citizennames', 'France')]
[('writername', 'Pierre Versins'), ('booknames', 'Encyclopedia of utopia, extraordinary journeys and science fiction'), ('occupationnames', 'essayist, publisher, science fiction writer, writer'), ('citizennames', 'France')]
[('writername', 'Manlio Cancogni'), ('booknames', 'Allegri, gioventù'), ('occupationnames', 'journalist, writer'), ('citizennames', 'Italy, Kingdom of Italy')]
[('writername', 

10

In [121]:
### Vatican Library. 

In [122]:

queryString = """
SELECT DISTINCT ?bookname ?writername ?publishedinname
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P6216 wd:Q19652 .
    ?book wdt:P1433 ?publishedin .
    ?book wdt:P577 ?time .
    FILTER("1810-01-01"^^xsd:dateTime <= ?time && ?time < "1910-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?publishedin <http://schema.org/name> ?publishedinname .

} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('bookname', 'Le Feu'), ('writername', "Gabriele D'Annunzio"), ('publishedinname', 'Revue de Paris')]
[('bookname', 'First Night'), ('writername', 'Luigi Pirandello'), ('publishedinname', '1900')]
[('bookname', 'A Turkish Peasant'), ('writername', 'Cristina Trivulzio Belgiojoso'), ('publishedinname', 'Revue des Deux Mondes')]


3

In [123]:
# all books by Kingdom of Italy+italy authors between 1810 until 1910 with their country of citizen + their copyright status.
# [('p', 'http://www.wikidata.org/prop/direct/P1433'), ('pname', 'published in')]
queryString = """
SELECT DISTINCT ?bookname ?writername ?publishedinname ?publishedin
WHERE { 
    {?writer wdt:P27 wd:Q38 }
    UNION { ?writer wdt:P27 wd:Q172579 } 
    ?writer wdt:P106 wd:Q36180 .
    
    ?book wdt:P50 ?writer .
    ?book wdt:P6216 wd:Q19652 .
    ?book wdt:P1433 ?publishedin .
    ?book wdt:P577 ?time .
    
    #?publishedin wdt:P195 wd:Q213678 . # collection
    #?publishedin wdt:P276 wd:Q213678 . #location
    #?publishedin wdt:P361 wd:Q213678 . #'part of'
    #?publishedin wdt:P485 wd:Q213678 . #archives at
    
    FILTER("1810-01-01"^^xsd:dateTime <= ?time && ?time < "1910-01-01"^^xsd:dateTime).
    #labels
    
    ?writer <http://schema.org/name> ?writername .
    ?book <http://schema.org/name> ?bookname .
    ?publishedin <http://schema.org/name> ?publishedinname .

} 
ORDER BY (?time)
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('bookname', 'A Turkish Peasant'), ('writername', 'Cristina Trivulzio Belgiojoso'), ('publishedinname', 'Revue des Deux Mondes'), ('publishedin', 'http://www.wikidata.org/entity/Q1569226')]
[('bookname', 'Le Feu'), ('writername', "Gabriele D'Annunzio"), ('publishedinname', 'Revue de Paris'), ('publishedin', 'http://www.wikidata.org/entity/Q1389508')]
[('bookname', 'First Night'), ('writername', 'Luigi Pirandello'), ('publishedinname', '1900'), ('publishedin', 'http://www.wikidata.org/entity/Q2034')]


3